# In this notebook several previously created models will be tested, namely:

#### Firstly, some models were trained on data without additional filtration

* Model trained on pairs: original english - simplified russian sentence. So, it learns both translate and simplify at the same time.
* Model trained only on the translated to Russian data.
* Model trained firstly on the original data and then on the translated corpus

### Then, as the results were not satisfying, an addsitional filtration and data was added

* Model trained on filtered translated data
* Model trained on filtered translated data and then on Paraphraser
* Model trained on Paraphraser
* Model trained on Paraphraser + filtered translated data
* Model trained on Paraphraser + filtered translated data with control tokens

All the models will be tested on the dev part of the dataset presented in RuSimpleSentEval competition. For the two of the models the russian dataset firstly will be translated into English.

## Necessary libraries

In [ ]:
! pip install googletrans
! pip install transformers
! pip install laserembeddings
! pip install sentence_transformers
! pip install language_tool_python
! pip install textstat

In [ ]:
! python -m laserembeddings download-models


✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


In [ ]:
import pandas as pd
import numpy as np
import torch
import language_tool_python
import re
import textstat
from laserembeddings import Laser
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from transformers import MarianMTModel, MarianTokenizer
from transformers.hf_api import HfApi
from googletrans import Translator
from tqdm import tqdm_notebook
import nltk
nltk.download('punkt')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Loading data...

In [ ]:
! mkdir data
! gdown https://drive.google.com/uc?id=1bJo8TagTGKa0uyppQRqsHrKHyYO5tcZc
! gdown https://drive.google.com/uc?id=11lqipq6ggrgCk8bVxQ4-uuPVMCKN5ebU
! gdown https://drive.google.com/uc?id=1dB3X-Wx8qU_5nDG_pxAmLvo5H_sgnHrE
! gdown https://drive.google.com/uc?id=1IVz3XC8Rm7hQCyx3xCcABjhaENrKABvF

In [ ]:
test_data = pd.read_csv('/content/test_baselines.csv', sep='\t')
test_data['INPUT:source'] = test_data['INPUT:source'].apply(lambda x: re.sub(r'[\t\n\r\f\v]+', "",x))
test_data['OUTPUT:output'] = test_data['OUTPUT:output'].apply(lambda x: re.sub(r'[\t\n\r\f\v]+', "",x))
test_data.to_csv('test_data_new.csv', index=False, sep='\t')

In [ ]:
data_train = pd.read_csv('/content/wiki_train_cleaned_translated_sd.csv')
data_dev = pd.read_csv('/content/wiki_dev_cleaned_translated_sd.csv')
data_test  = pd.read_csv('/content/test_data_new.csv',  sep='\t')

In [ ]:
data_test.sample(5)

,Unnamed: 0,INPUT:source,OUTPUT:output,trunctuation_bs,gpt_bs,cos_sim_ref,cos_sim_tr_bs,cos_sim_gpt_bs
1141,3356,"Здесь он пишет «Письмо Онегина», тем самым окончательно завершая работу над романом в стихах, который был его «спутником верным» на протяжении восьми лет жизни.","Здесь он написал ""Письмо Онегина"" и этим закончил поэму, которую писал 8 лет.","Здесь он пишет «Письмо Онегина», тем самым окончательно завершая работу над романом в стихах,.","Здесь он пишет «Письмо Онегина», тем самым окончательно завершая работу о романе в 1843 году.",0.790754,0.844982,0.838458
314,1105,"В настоящее время не существует лекарств, способных обратить или хотя бы замедлить развитие болезни Альцгеймера.","Сейчас нет лекарств, которые могли хотя бы затормозить развитие болезни Альцгеймера.","В настоящее время не существует лекарств, способных обратить или.","В настоящее время не существует лекарств, способных обратить или хотя бы замедлить развитие болезни.",0.988584,0.985970,0.994800
1227,3610,Имперский камерный суд состоял теперь в равной пропорции из католиков и протестантов и должен был стать основным инструментом мирного урегулирования.,Имперский камерный суд теперь составляют католики и протестанты.,Имперский камерный суд состоял теперь в равной пропорции из католиков и протестантов.,"Имперский камерный суд состоял теперь в равной пропорции из католиков и протестантов, которые были равны между собой как по численности населения (апрель 1797 года), так даже для их веры.",0.951733,0.977822,0.750965
351,1181,В основе заболевания лежит ускорение инактивации инсулина или же специфическое разрушение рецепторов инсулина на мембранах инсулинзависимых клеток.,Причина заболевания - снижение инсулина или разрешение рецептеров инсулина на клетки;,В основе заболевания лежит ускорение инактивации инсулина или же специфическое.,В основе заболевания лежит ускорение инактивации инсулина или же специфическое повышение иммунитета в крови.,0.957886,0.966153,0.975032
914,2698,"Далее, азербайджанские войска начали наступление по нескольким направлениям с целью подчинить над армянскими сёлами Нагорного Карабаха.",войска азербайджана начали наступление с целью захвата Нагорного Карабаха,"Далее, азербайджанские войска начали наступление по нескольким направлениям с.","Далее, азербайджанские войска начали наступление по нескольким направлениям с целью подчинить над армянской армией Азербайджан и его армию в конечном итоге.",0.978669,0.981258,0.928678


## Setting the environment for Fairseq mBart

In [ ]:
! wget https://dl.fbaipublicfiles.com/fairseq/models/mbart/mbart.cc25.v2.tar.gz
! tar -xzvf /content/mbart.cc25.v2.tar.gz
! apt-get install cmake build-essential pkg-config libgoogle-perftools-dev

In [ ]:
! git clone https://github.com/google/sentencepiece.git 
%cd sentencepiece
! mkdir build

In [ ]:
%cd build
!cmake ..
!make
!make install
!ldconfig -v

In [ ]:
%cd /content

/content


In [ ]:
! git clone https://github.com/pytorch/fairseq
%cd /content/fairseq/
! python -m pip install --editable .
%cd /content

! echo $PYTHONPATH

import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"

! echo $PYTHONPATH


In [ ]:
! git clone https://github.com/feralvam/easse
! git clone https://github.com/Andoree/sent_simplification.git
%cp /content/sent_simplification/sari.py /content/easse/easse

%cd easse
! pip install .

remote: Enumerating objects: 1888, done.
remote: Counting objects: 100% (455/455), done.
remote: Compressing objects: 100% (282/282), done.
remote: Total 1888 (delta 260), reused 312 (delta 167), pack-reused 1433
Receiving objects: 100% (1888/1888), 33.78 MiB | 3.55 MiB/s, done.
Resolving deltas: 100% (1133/1133), done.
Cloning into 'sent_simplification'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 37 (delta 18), reused 8 (delta 2), pack-reused 0
Unpacking objects: 100% (37/37), done.
/content/easse
Processing /content/easse
     |████████████████████████████████| 1.5MB 14.0MB/s 
     |████████████████████████████████| 61kB 8.6MB/s 
     |████████████████████████████████| 163kB 63.4MB/s 
     |████████████████████████████████| 61kB 8.5MB/s 
  Cloning https://github.com/facebookresearch/text-simplification-evaluation.git to /tmp/pip-install-i8onzbas/tseval
  Running command git clon

In [ ]:
%cd /content

/content


In [ ]:
! mkdir prepared_data
! mkdir preds

## Translation to English

We have to translate data to English to test the english models and also the model trained on english source and russian target sentences

In [ ]:
data_test = data_test[['Unnamed: 0', 'INPUT:source','OUTPUT:output', 'trunctuation_bs', 'gpt_bs']]

In [ ]:
# helper functions

def batch_generator(
        list_of_sentences,
        size=64
):
    num_batch = len(list_of_sentences)//size
    for index in range(num_batch):
        yield list_of_sentences[index*size:(index+1)*size]
    yield list_of_sentences[num_batch*size:]


def translate_to_eng(model, tok, src, dst):

  translations_src = []
  for i in tqdm_notebook(batch_generator(src)):
    l = tok(i, return_tensors="pt", padding=True).to(device)
    translated = model.generate(**l)
    translations_src.extend([tok.decode(t, skip_special_tokens=True) for t in translated])

  translations_dst = []
  for i in tqdm_notebook(batch_generator(dst)):
    l = tok(i, return_tensors="pt", padding=True).to(device)
    translated = model.generate(**l)
    translations_dst.extend([tok.decode(t, skip_special_tokens=True) for t in translated])

  return translations_src, translations_dst

In [ ]:
# import en-ru model from transformers
model_name = 'Helsinki-NLP/opus-mt-ru-en'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
model.to(device)

data_test['INPUT:source'] = data_test['INPUT:source'].astype(str)
data_test['OUTPUT:output'] = data_test['OUTPUT:output'].astype(str)

In [ ]:
# test--------------------------------------------------
src, dst = translate_to_eng(model, tokenizer, list(data_test['INPUT:source'].values), list(data_test['OUTPUT:output'].values))
for i, obj  in enumerate(src):
  src[i] = src[i].replace('&gt;', '')
  #src[i] = re.sub(r'[&gt;{1,}]', " ", src[i])
  src[i] = src[i].replace('&lt;', '')
  src[i] = src[i].replace('()', '')
  src[i] = src[i].replace('&quot;', '') 
  src[i] = src[i].replace('&apos;', '')
for i, obj in enumerate(dst):
  dst[i] = dst[i].replace('&gt;', '')
  dst[i] = dst[i].replace('&lt;', '')
  dst[i] = dst[i].replace('()', '')
  dst[i] = dst[i].replace('&quot;', '')
  dst[i] = dst[i].replace('&apos;', '')
data_test['src'] = src
data_test['dst'] = dst
data_test.to_csv('/content/drive/MyDrive/MT_sentence_simpl/wiki_test_dev_eng.csv', sep='\t')

In [ ]:
data_test.sample(5)

,Unnamed: 0,INPUT:source,OUTPUT:output,trunctuation_bs,gpt_bs,src,dst
1021,3120,"Если нельзя установить, какая фигура была тронута первой, считается, что это фигура игрока, за которым очередь хода, а не его партнёра.","Если неизвестно, какая фигура была тронута первой, считается, что это фигура того игрока, которой должен ходить.","Если нельзя установить, какая фигура была тронута первой, считается, что это фигура.","Если нельзя установить, какая фигура была тронута первой, считается, что это фигура «неправильная».","If it is not possible to determine which figure was touched by the first one, it is believed to be the figure of the player behind whom the turn of the move is not his partner.","If it is not known which figure was touched by the first one, it is believed to be the figure of the player that should walk."
2001,5984,"Оставленный в Кинбурне гарнизон не успел до холодов восстановить сгоревшие в результате бомбардировки казармы, поэтому жил в землянках.",Гарнизон Кинбурга не смог до наступления зимы отстроить сгоревшие после бомбардировки казармы. Поэтому солдатам пришлось разместится в вырытых землянках.,Оставленный в Кинбурне гарнизон не успел до холодов восстановить сгоревшие.,"Оставленный в Кинбурне гарнизон не успел до холодов восстановить сгоревшие постройки и деревни, а также несколько других деревень.","The garrison left in Kinbourne was not able to recover the burning of the barracks, so it lived in the ground.","The Kinburg Garnizon was unable until winter to rebuild the burning barracks after the bombing, so the soldiers had to sit in dug-out strawberries."
602,1873,Вариант интерпретатора с синхронизацией доступа к отдельным объектам вместо глобальной блокировки из-за частых захватов/освобождений блокировок оказался слишком медленным.,Вариант доступа к отдельным объектам оказался слишком медленным,Вариант интерпретатора с синхронизацией доступа к отдельным объектам вместо глобальной.,"Вариант интерпретатора с синхронизацией доступа к отдельным объектам вместо глобальной блокировки в сети позволяет получить доступ только из одной области, например.",The interpretation option of synchronizing access to individual objects instead of global blockage due to frequent captures/releases of blocks has been too slow.,The option of access to individual facilities proved to be too slow
19,71,"А по размаху политических репрессий против инакомыслящих хрущёвский режим стоит несоизмеримо ближе к Сталину, чем брежневский.","Политический режим Хрущёва больше похож на режим Сталина, чем режим Брежнева.",А по размаху политических репрессий против инакомыслящих хрущёвский режим.,"А по размаху политических репрессий против инакомыслящих хрущёвский СССР был третьим советским государством, состоявшим из пяти округов.",And by the scale of the political repression against the dissenting Kraushkov regime is disproportionately closer to Stalin than the Brezhnevian regime.,Chrushchev's political regime is more like Stalin's regime than Brezhnev's.
2581,7658,"Различные же теории заговоров создают атмосферу страха, слухи и предрассудки, подобные теории осуждаются научным сообществом.",Найчное сообщества и рассуждения о теориях и их влияния на разум .,"Различные же теории заговоров создают атмосферу страха, слухи и.","Различные же теории заговоров создают атмосферу страха, слухи и предрассудки, подобные тому что говорят.","Various conspiracy theories, however, create an atmosphere of fear, rumours and prejudice, such theories being condemned by the scientific community.",Scientific communities and discourse about theories and their influence on the mind.


In [ ]:
# data_test = pd.read_csv('/content/drive/MyDrive/MT_sentence_simpl/wiki_test_dev_eng.csv', sep='\t')

## Inference stage

All the models will be evaluated according to the following aspects:

* SARI
* BLEU
* FKGL
* Grammar aspect

All the predictions were saved in advance. If there is a need to make a new prediction, mBART model can be loaded but it requires much RAM:(
   Also, it is necessary to make the following change in /content/fairseq/fairseq/tasks/translation_from_pretrained_bart.py:

```
def __init__(self, args, src_dict, tgt_dict):
        super().__init__(args, src_dict, tgt_dict)
        self.args = args                  # add this line !!!!!
        self.langs = args.langs.split(",")
        for d in [src_dict, tgt_dict]:
            for l in self.langs:
```


In [ ]:
! mkdir /content/fairseq/data

In [ ]:
### process WikiLarge but with Russian test
with open('/content/fairseq/data/test.src', "a") as f:
  for i, row in data_test.iterrows():
    f.write(row['INPUT:source']+'\n')

with open('/content/fairseq/data/train.src', "a") as f:
  for i, row in data_train.iterrows():
    f.write(row['src']+'\n')

with open('/content/fairseq/data/dev.src', "a") as f:
  for i, row in data_dev.iterrows():
    f.write(row['src']+'\n')

with open('/content/fairseq/data/test.dst', "a") as f:
  for i, row in data_test.iterrows():
    f.write(row['OUTPUT:output']+'\n')

with open('/content/fairseq/data/train.dst', "a") as f:
  for i, row in data_train.iterrows():
    f.write(row['dst']+'\n')

with open('/content/fairseq/data/dev.dst', "a") as f:
  for i, row in data_dev.iterrows():
    f.write(row['dst']+'\n')


In [ ]:
#### process translated WikiLarge + russian dev set as test
with open('/content/fairseq/data/test.src', "a") as f:
  for i, row in data_test.iterrows():
    f.write(row['INPUT:source']+'\n')

with open('/content/fairseq/data/train.src', "a") as f:
  for i, row in data_train.iterrows():
    f.write(row['target_x']+'\n')

with open('/content/fairseq/data/dev.src', "a") as f:
  for i, row in data_dev.iterrows():
    f.write(row['target_x']+'\n')

with open('/content/fairseq/data/test.dst', "a") as f:
  for i, row in data_test.iterrows():
    f.write(row['OUTPUT:output']+'\n')

with open('/content/fairseq/data/train.dst', "a") as f:
  for i, row in data_train.iterrows():
    f.write(row['target_y']+'\n')

with open('/content/fairseq/data/dev.dst', "a") as f:
  for i, row in data_dev.iterrows():
    f.write(row['target_y']+'\n')

In [ ]:
SPM="/content/sentencepiece/build/src/spm_encode"
BPE_MODEL="/content/mbart.cc25.v2/sentence.bpe.model"
DATA_DIR="/content/fairseq/data"
SRC="src"
TGT="dst" #en

!$SPM --model=$BPE_MODEL < $DATA_DIR/train.$SRC > $DATA_DIR/train.spm.$SRC &
!$SPM --model=$BPE_MODEL < $DATA_DIR/train.$TGT > $DATA_DIR/train.spm.$TGT &
!$SPM --model=$BPE_MODEL < $DATA_DIR/dev.$SRC > $DATA_DIR/dev.spm.$SRC &
!$SPM --model=$BPE_MODEL < $DATA_DIR/dev.$TGT > $DATA_DIR/dev.spm.$TGT &
!$SPM --model=$BPE_MODEL < $DATA_DIR/test.$SRC > $DATA_DIR/test.spm.$SRC &
!$SPM --model=$BPE_MODEL < $DATA_DIR/test.$TGT > $DATA_DIR/test.spm.$TGT &

In [ ]:

PREPROCESSED_DATA_DIR="/content/fairseq/data"
DICT="/content/mbart.cc25.v2/dict.txt"
!fairseq-preprocess \
  --source-lang src \
  --target-lang dst \
  --trainpref /content/fairseq/data/train.spm \
  --validpref /content/fairseq/data/dev.spm \
  --testpref /content/fairseq/data/test.spm \
  --destdir /content/fairseq/data \
  --thresholdtgt 0 \
  --thresholdsrc 0 \
  --srcdict /content/mbart.cc25.v2/dict.txt \
  --tgtdict /content/mbart.cc25.v2/dict.txt \
  --workers 70

2021-05-01 00:55:26 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='/content/fairseq/data', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=False, scoring='bleu', seed=1, source_lang='src', srcdict='/content/mbart.cc25.v2/dict.txt', suppress_crashes=False, target_lang='dst', task='translation', tensorboard_logdir=None, testpref='/content/fairseq/data

In [ ]:
! pip install sentencepiece

In [ ]:
!fairseq-generate /content/fairseq/data \
  --path  /content/drive/MyDrive/checkpoints_ru_ru_par22/checkpoint_best.pt \
  --task translation_from_pretrained_bart \
  --gen-subset test \
  --source-lang src --target-lang dst \
  --bpe 'sentencepiece' --sentencepiece-model /content/mbart.cc25.v2/sentence.bpe.model \
  --sacrebleu --remove-bpe 'sentencepiece' \
  --batch-size 32 --langs ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN > model_prediction.txt & 
!cat model_prediction.txt | grep -P "^H" |sort -V |cut -f 3- > model_prediction_ru_par22_filtered15.hyp


In [ ]:
! cp /content/model_prediction_ru_par22_filtered15.hyp /content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_ru22_par_filtered15.hyp

In [ ]:
! python /content/sent_simplification/refs_to_easse_format.py \
--input_path /content/drive/MyDrive/MT_sentence_simpl/wiki_test_dev_eng.csv \
--output_dataset_name test_ref_data \
--src_column "INPUT:source" \
--trg_column "OUTPUT:output" \
--output_dir /content/prepared_data



1000
3406
3406
Overall number of references: 3406


In [ ]:
with open('/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_ru22_par_filtered15.hyp', 'r') as f:
  sentences = [i.strip() for i in f.readlines()]

lt = list()
st = set()
for i in sentences:
  if i not in st:
    lt.append(i)
    st.add(i)

with open('/content/model_prediction_ru22_par_filtered15_test.hyp', 'w') as f:
  for i in lt:
    f.write(i+'.\n')


In [ ]:
! easse evaluate \
--test_set custom \
--metrics sari \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction_ru22_par_filtered15_test.hyp -q

[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
{'sari': 34.738, 'quality_estimation': {'Compression ratio': 0.632, 'Sentence splits': 0.986, 'Levenshtein similarity': 0.721, 'Exact copies': 0.039, 'Additions proportion': 0.045, 'Deletions proportion': 0.418, 'Lexical complexity score': 10.734}}


## Evaluation functions

BLEU

In [ ]:
! pip install sacrebleu

In [ ]:
# function retrieved from easse code
import numpy as np
from typing import List
import sacrebleu
import easse.utils.preprocessing as utils_prep
from collections import OrderedDict


def corpus_bleu(
    sys_sents: List[str],
    refs_sents: List[List[str]],
    smooth_method: str = 'exp',
    smooth_value: float = None,
    force: bool = True,
    lowercase: bool = False,
    tokenizer: str = '13a',
    use_effective_order: bool = False,
):

    sys_sents = [utils_prep.normalize(sent, lowercase, tokenizer) for sent in sys_sents]
    refs_sents = [[utils_prep.normalize(sent, lowercase, tokenizer) for sent in ref_sents] for ref_sents in refs_sents]

    return sacrebleu.corpus_bleu(
        sys_sents,
        refs_sents,
        smooth_method,
        smooth_value,
        force,
        lowercase=False,
        tokenize='none',
        use_effective_order=use_effective_order,
    ).score


def sentence_bleu(
    sys_sent: str,
    ref_sents: List[str],
    smooth_method: str = 'floor',
    smooth_value: float = None,
    lowercase: bool = False,
    tokenizer: str = '13a',
    use_effective_order: bool = True,
):

    return corpus_bleu(
        [sys_sent],
        [[ref] for ref in ref_sents],
        smooth_method,
        smooth_value,
        force=True,
        lowercase=lowercase,
        tokenizer=tokenizer,
        use_effective_order=use_effective_order,
    )


def corpus_averaged_sentence_bleu(
    sys_sents: List[str],
    refs_sents: List[List[str]],
    smooth_method: str = 'floor',
    smooth_value: float = None,
    lowercase: bool = False,
    tokenizer: str = '13a',
    use_effective_order: bool = True,
):

    scores = []
    for sys_sent, *ref_sents in zip(sys_sents, *refs_sents):
        scores.append(
            sentence_bleu(
                sys_sent,
                ref_sents,
                smooth_method,
                smooth_value,
                lowercase=lowercase,
                tokenizer=tokenizer,
                use_effective_order=use_effective_order,
            )
        )
    return np.mean(scores)


### Flesch Kincaid Grade Level

In [ ]:
# https://github.com/infoculture/plainrussian
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from math import sqrt
import csv


from numpy import mean, arange


# Russian sounds and characters
RU_CONSONANTS_LOW = [u'к', u'п', u'с', u'т', u'ф', u'х', u'ц', u'ч', u'ш', u'щ']
RU_CONSONANTS_HIGH = [u'б', u'в', u'г', u'д', u'ж', u'з']
RU_CONSONANTS_SONOR = [u'л', u'м', u'н', u'р']
RU_CONSONANTS_YET = [u'й']

RU_CONSONANTS = RU_CONSONANTS_HIGH + RU_CONSONANTS_LOW + RU_CONSONANTS_SONOR + RU_CONSONANTS_YET
RU_VOWELS = [u'а', u'е', u'и', u'у', u'о', u'я', u'ё', u'э', u'ю', u'я', u'ы']
RU_MARKS = [u'ь', u'ъ']
SENTENCE_SPLITTERS = [u'.', u'?', u'!']
RU_LETTERS = RU_CONSONANTS + RU_MARKS + RU_VOWELS
SPACES = [u' ', u'\t']

# List of prepared texts

GRADE_TEXT = {
    1: u'1 - 3-й класс (возраст примерно: 6-8 лет)',
    2: u'1 - 3-й класс (возраст примерно: 6-8 лет)',
    3: u'1 - 3-й класс (возраст примерно: 6-8 лет)',
    4: u'4 - 6-й класс (возраст примерно: 9-11 лет)',
    5: u'4 - 6-й класс (возраст примерно: 9-11 лет)',
    6: u'4 - 6-й класс (возраст примерно: 9-11 лет)',
    7: u'7 - 9-й класс (возраст примерно: 12-14 лет)',
    8: u'7 - 9-й класс (возраст примерно: 12-14 лет)',
    9: u'7 - 9-й класс (возраст примерно: 12-14 лет)',
    10: u'10 - 11-й класс (возраст примерно: 15-16 лет)',
    11: u'10 - 11-й класс (возраст примерно: 15-16 лет)',
    12: u'1 - 3 курсы ВУЗа (возраст примерно: 17-19 лет)',
    13: u'1 - 3 курсы ВУЗа (возраст примерно: 17-19 лет)',
    14: u'1 - 3 курсы ВУЗа (возраст примерно: 17-19 лет)',
    15: u'4 - 6 курсы ВУЗа (возраст примерно: 20-22 лет)',
    16: u'4 - 6 курсы ВУЗа (возраст примерно: 20-22 лет)',
    17: u'4 - 6 курсы ВУЗа (возраст примерно: 20-22 лет)',
}

POST_GRADE_TEXT_18_24 = u'Аспирантура, второе высшее образование, phD'


def calc_SMOG(n_psyl, n_sent):
    """Метрика SMOG для английского языка"""
    n = 1.0430 * sqrt((float(30.0) / n_sent) * n_psyl) + 3.1291
    return n

def calc_Gunning_fog(n_psyl, n_words, n_sent):
    """Метрика Gunning fog для английского языка"""
    n = 0.4 * ((float(n_words)/ n_sent) + 100 * (float(n_psyl) / n_words))
    return n

def calc_Dale_Chale(n_psyl, n_words, n_sent):
    """Метрика Dale Chale для английского языка"""
    n = 0.1579 * (100.0 * n_psyl / n_words) + 0.0496 * (float(n_words) / n_sent)
    return n

def calc_Flesh_Kincaid(n_syllabes, n_words, n_sent):
    """Метрика Flesh Kincaid для английского языка"""
    n = 206.835 - 1.015 * (float(n_words) / n_sent) - 84.6 * (float(n_syllabes) / n_words)
    return n


def calc_Flesh_Kincaid_rus(n_syllabes, n_words, n_sent):
    """Метрика Flesh Kincaid для русского языка"""
    n = 220.755 - 1.315 * (float(n_words) / n_sent) - 50.1 * (float(n_syllabes) / n_words)
    return n


# The variant of the FC formula for the Russian language is as follows:FK = (0.5 x average sentence length) +
# (8.4 x average number of syllables in a word) - 15.59

# def calc_Flesh_Kincaid_Grade_rus(n_syllabes, n_words, n_sent):
#     """Метрика Flesh Kincaid Grade для русского языка"""
# #    n = 0.59 * (float(n_words) / n_sent) + 6.2 * (float(n_syllabes) / n_words) - 16.59
#     n = 0.49 * (float(n_words) / n_sent) + 7.3 * (float(n_syllabes) / n_words) - 16.59
#     return n

def calc_Flesh_Kincaid_Grade_rus(n_syllabes, n_words, n_sent):
    """Метрика Flesh Kincaid Grade для русского языка"""
#    n = 0.59 * (float(n_words) / n_sent) + 6.2 * (float(n_syllabes) / n_words) - 16.59
    n = 0.5 * (float(n_words) / n_sent) + 8.4 * (float(n_syllabes) / n_words) - 15.59
    return n



def calc_Flesh_Kincaid_Grade_rus_adapted(n_syllabes, n_words, n_sent, X, Y, Z):
    """Метрика Flesh Kincaid Grade для русского языка с параметрами"""
#    n = 0.59 * (float(n_words) / n_sent) + 6.2 * (float(n_syllabes) / n_words) - 16.59
    if n_words == 0 or n_sent == 0: return 0
    n = X * (float(n_words) / n_sent) + Y * (float(n_syllabes) / n_words) - Z
    return n


#X_GRADE = 0.186
#Y_GRADE = 7.21
#Z_GRADE = 15.443

# Flesh Kinkaid Grade константы. Подробнее http://en.wikipedia.org/wiki/Flesch%E2%80%93Kincaid_readability_tests
FLG_X_GRADE = 0.318
FLG_Y_GRADE = 14.2
FLG_Z_GRADE = 30.5

def calc_Flesh_Kincaid_Grade_rus_flex(n_syllabes, n_words, n_sent):
    """Метрика Flesh Kincaid Grade для русского языка с константными параметрами"""
    if n_words == 0 or n_sent == 0: return 0
    n = FLG_X_GRADE * (float(n_words) / n_sent) + FLG_Y_GRADE * (float(n_syllabes) / n_words) - FLG_Z_GRADE
    return n


# Coleman Liau константы. Подробнее http://en.wikipedia.org/wiki/Coleman%E2%80%93Liau_index

CLI_X_GRADE = 0.055
CLI_Y_GRADE = 0.35
CLI_Z_GRADE = 20.33


def calc_Coleman_Liau_index_adapted(n_letters, n_words, n_sent, x, y, z):
    """ Метрика Coleman Liau для русского языка с адаптированными параметрами """
    if n_words == 0: return 0
    n = x * (n_letters * (100.0 / n_words)) - y * (n_sent * (100.0 / n_words)) - z
    return n

def calc_Coleman_Liau_index(n_letters, n_words, n_sent):
    """ Метрика Coleman Liau для русского языка с константными параметрами """
    if n_words == 0: return 0
    n = CLI_X_GRADE * (n_letters * (100.0 / n_words)) - CLI_Y_GRADE * (n_sent * (100.0 / n_words)) - CLI_Z_GRADE
    return n


# Константы SMOG Index http://en.wikipedia.org/wiki/SMOG
SMOG_X_GRADE = 1.1
SMOG_Y_GRADE = 64.6
SMOG_Z_GRADE = 0.05

def calc_SMOG_index(n_psyl, n_sent):
    """Метрика SMOG для русского языка с константными параментрами"""
    n = SMOG_X_GRADE * sqrt((float(SMOG_Y_GRADE) / n_sent) * n_psyl) + SMOG_Z_GRADE
    return n

def calc_SMOG_index_adapted(n_psyl, n_sent, x, y, z):
    """Метрика SMOG для русского языка адаптированная с коэффициентами"""
    n = x * sqrt((float(y) / n_sent) * n_psyl) + z
    return n

DC_X_GRADE = 0.552
DC_Y_GRADE = 0.273

def calc_Dale_Chale_index(n_psyl, n_words, n_sent):
    """Метрика Dale Chale для русского языка с константным параметрами"""
    n = DC_X_GRADE * (100.0 * n_psyl / n_words) + DC_Y_GRADE * (float(n_words) / n_sent)
    return n


def calc_Dale_Chale_adapted(n_psyl, n_words, n_sent, x, y):
    """Метрика Dale Chale для русского языка с адаптированными параметрами"""
    n = x * (100.0 * n_psyl / n_words) + y * (float(n_words) / n_sent)
    return n

ARI_X_GRADE = 6.26
ARI_Y_GRADE = 0.2805
ARI_Z_GRADE = 31.04


def calc_ARI_index_adapted(n_letters, n_words, n_sent, x, y, z):
    """ Метрика Automated Readability Index (ARI) для русского языка с адаптированными параметрами """
    if n_words == 0 or n_sent == 0: return 0
    n = x * (float(n_letters) / n_words) + y * (float(n_words) / n_sent) - z
    return n

def calc_ARI_index(n_letters, n_words, n_sent):
    """ Метрика Automated Readability Index (ARI) для русского языка с константными параметрами """
    if n_words == 0 or n_sent == 0: return 0
    n = ARI_X_GRADE * (float(n_letters) / n_words) + ARI_Y_GRADE * (float(n_words) / n_sent) - ARI_Z_GRADE
    return n


def load_words(filename):
    """Load words from filename"""
    words = []
    f = open(filename, 'r')
    for l in f:
        words.append(l.strip().decode('utf8'))
    f.close()
    return words

#FAM_WORDS = load_words('1norm50000.txt')

bad_chars = '(){}<>"\'!?,.:;'


def calc_text_metrics(filename, verbose=True):
    """Расчет метрик"""
    f = open(filename, 'r')
    text = f.read().decode('utf8')    
    f.close()
    return calc_readability_metrics(text, verbose)


# Number of syllabes for long words
COMPLEX_SYL_FACTOR = 4

def calc_readability_metrics(text, verbose=True):
    sentences = 0
    chars = 0
    spaces = 0
    letters = 0
    syllabes = 0
    words = 0
    complex_words = 0
    simple_words = 0
    wsyllabes = {}

    wordStart = False
    for l in text.splitlines():
        chars += len(l)
#        l = l.decode('utf8')
        for ch in l:
            if ch in SENTENCE_SPLITTERS:
                sentences += 1
            if ch in SPACES:
                spaces += 1

        for w in l.split():
            has_syl = False
            has_letter = False #ADDED
            wsyl = 0
#            if len(w) > 1: words += 1
            for ch in w:
                if ch in RU_LETTERS:
                    letters += 1
                    has_letter = True #ADDED
                if ch in RU_VOWELS:
                    syllabes += 1
                    has_syl = True
                    wsyl += 1
            if wsyl > COMPLEX_SYL_FACTOR:
                complex_words += 1
            elif wsyl < COMPLEX_SYL_FACTOR+1 and wsyl > 0:
                simple_words += 1
            if has_syl or has_letter: #ADDED
                words += 1
            if has_syl:
                # words += 1 #CHANGED
                v = wsyllabes.get(str(wsyl), 0)
                wsyllabes[str(wsyl)] = v + 1
    metrics = {'c_share': float(complex_words) * 100 / words if words > 0 else 0,
               'avg_slen' : float(words) / sentences if sentences > 0 else 0,
               'avg_syl' : float(syllabes) / words if words > 0 else 0,
               'n_syllabes': syllabes,
               'n_words' : words,
               'n_sentences': sentences,
               'n_complex_words': complex_words,
               'n_simple_words' : simple_words,
               'chars': chars,
               'letters' : letters,
               'spaces' : spaces,
               'index_fk_rus': calc_Flesh_Kincaid_Grade_rus_flex(syllabes, words, sentences),  #calc_Flesh_Kincaid_Grade_rus_flex
               'index_cl_rus' : calc_Coleman_Liau_index(letters, words, sentences),
               'index_dc_rus' : calc_Dale_Chale_index(complex_words, words, sentences),
               'index_SMOG_rus' : calc_SMOG_index(complex_words, sentences),
               'index_ari_rus' : calc_ARI_index(letters, words, sentences),
#               'index_fk_rus': calc_Flesh_Kincaid_Grade_rus(syllabes, words, sentences),
               'wsyllabes' : wsyllabes
    }
    del text
    return metrics

#### COSINE SIMILARITY

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# shape should be [1, something (768, ex)]

# import numpy as np
# def cs(a, b):
#   return (a @ b.T)/(np.linalg.norm(a)*np.linalg.norm(b))

def calc_cos_sim(df, model,tok, x, y, column_name):
    Cos_sim= []
    for index, row in df.iterrows():
        
        # original
          sentence_A = tok.encode(row[x], padding='max_length', max_length=50, truncation=True, return_tensors='pt')
          sentence_A = sentence_A.to(device)
          output = model(sentence_A)
          sent_emb = output[-1][0]
          emb_source = sent_emb.mean(axis=1)
          emb_source = emb_source.cpu().detach().numpy()

          sentence_B = tok.encode(row[y], padding='max_length', max_length=50, truncation=True, return_tensors='pt')
          sentence_B = sentence_B.to(device)
          output = model(sentence_B)
          sent_emb = output[-1][0]
          emb_target= sent_emb.mean(axis=1)
          emb_target = emb_target.cpu().detach().numpy()

          cos_val = cosine_similarity(emb_source.reshape(emb_source.shape[0], -1), emb_target.reshape(emb_target.shape[0], -1))[0][0]
          Cos_sim.append(cos_val)
    df[column_name] = Cos_sim

GRAMMAR

In [ ]:
def get_mistakes_summary(df_test, x):
    test = list(df_test[x].values)
    matches = []
    for i in test:
      matches.extend(tool.check(i))

    categories = set([i.category for i in matches])

    categories = {i:0 for i in categories}

    for i in matches:
      categories[i.category]+=1

    return categories

Add necessary predictions to the dataframe

In [ ]:
with open("/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_en_en.hyp", 'r') as f:
  lines = [i.strip() for i in f.readlines()]
data_test['en_en'] = lines

with open("/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_en_ru.hyp", 'r') as f:
  lines = [i.strip() for i in f.readlines()]
data_test['en_ru'] = lines

with open("/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_ru_ru.hyp", 'r') as f:
  lines = [i.strip() for i in f.readlines()]
data_test['ru_ru'] = lines

with open("/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_en_ru_ru.hyp", 'r') as f:
  lines = [i.strip() for i in f.readlines()]
data_test['en_ru_ru'] = lines


# filtered data---------------------------
with open("/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_en_en_filtered.hyp", 'r') as f:
  lines = [i.strip() for i in f.readlines()]
data_test['en_en_filtered'] = lines

with open("/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_ru_ru_filtered.hyp", 'r') as f:
  lines = [i.strip() for i in f.readlines()]
data_test['ru_ru_filtered'] = lines

with open("/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_en_ru_ru_filtered.hyp", 'r') as f:
  lines = [i.strip() for i in f.readlines()]
data_test['en_ru_ru_filtered'] = lines

with open("/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_paraphraser2.hyp", 'r') as f:
  lines = [i.strip()+'.' for i in f.readlines()]
data_test['paraphraser'] = lines

with open("/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_paraphraser_ru2_12epochs.hyp", 'r') as f:
  lines = [i.strip() for i in f.readlines()]
data_test['paraphraser_ru'] = lines


with open("/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_ru22_par_filtered15.hyp", 'r') as f:
  lines = [i.strip()+'.' for i in f.readlines()]
data_test['ru_paraphraser'] = lines

with open("/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction__ru_ru_filtered15.hyp", 'r') as f:
  lines = [i.strip() for i in f.readlines()]
data_test['ru_15'] = lines

with open("/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_wiki_15_filtered_epochs.hyp", 'r') as f:
  lines = [i.strip() for i in f.readlines()]
data_test['ru_+_paraphraser'] = lines

with open("/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_con_tok_38sari.hyp", 'r') as f:
  lines = [i.strip() for i in f.readlines()]
data_test['con_tok'] = lines

In [ ]:
data_test.head()

,Unnamed: 0,INPUT:source,OUTPUT:output,trunctuation_bs,gpt_bs,en_en,en_ru,ru_ru,en_ru_ru,ru_ru_filtered,en_ru_ru_filtered,paraphraser,paraphraser_ru
0,3,"14 декабря 1944 года рабочий посёлок Ички был переименован в рабочий посёлок Советский, после чего поселковый совет стал называться Советским.",14 декабря 1944 года рабочий посёлок Ички переименован в Советский.,14 декабря 1944 года рабочий посёлок Ички был переименован в рабочий посёлок.,"14 декабря 1944 года рабочий посёлок Ички был переименован в рабочий посёлок Советский, который тогда назывался Рабоче-Крестьянской республикой.","On December 14 , 1944 , Ichka was renamed the Soviet worktown , after which the village council became the Soviet town .","14 декабря 1944 года рабочий город Ика был переименован в советский рабочий город, после чего деревня стала советским городом.","14 декабря 1944 года рабочий посёлок Ички был переименован в рабочий посёлок Советский, после чего поселковый совет стал называться Советским.","14 декабря 1944 года рабочий посёлок Ички был переименован в рабочий посёлок Советский, после чего поселковый совет стал называться Советским.",14 декабря 1944 года рабочий посёлок Ички был переименован в рабочий посёлок Советский.,14 декабря 1944 года рабочий посёлок Ички был переименован в рабочий посёлок Советский.,В Ичке рабочий посёлок переименован в рабочий посёлок Советский.,14 декабря 1944 года рабочий посёлок Ички был переименован в рабочий посёлок Советский.
1,4,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.,"В 1960 году вышла модель 172А. Отличие в хвостовом оперении, в руле направления и в креплении шасси",1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления.,1960 году была выпущена модель 172A. Изменения: хвостовое оперение было изменено на форму головы.,Model 172A was released in 1960 .,Модель 172A была выпущена в 1960 году.,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.,Изменения: хвостовое оперение и руль направления с обратной стреловидностью.,Изменения: хвостовое оперение и руль направления с обратной стреловидностью.,Модель 172A была выпущена в 1960 году.,В 1960 году была выпущена модель 172A.
2,5,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.,В выпущенной в 1960 году модель имела изменения в хвостовом оперении и руле направления,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления.,"1960 году была выпущена модель 172A. Изменения: хвостовое оперение, крылья и все другие детали сделали его менее приспособленным к жизни.",Model 172A was released in 1960 .,Модель 172A была выпущена в 1960 году.,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.,Изменения: хвостовое оперение и руль направления с обратной стреловидностью.,Изменения: хвостовое оперение и руль направления с обратной стреловидностью.,Модель 172A была выпущена в 1960 году.,В 1960 году была выпущена модель 172A.
3,6,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.,Изменения: в хвосте и руле направления с обратной стреловидностью и крепления для поплавкового шасси.,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления.,1960 году была выпущена модель 172A. Изменения: хвостовое оперение было заменено н

## Evaluation of English model trained on data without filtering

### SARI

In [ ]:
! python /content/sent_simplification/refs_to_easse_format.py \
--input_path /content/drive/MyDrive/MT_sentence_simpl/wiki_test_dev_eng.csv \
--output_dataset_name test_ref_data \
--src_column "src" \
--trg_column "dst" \
--output_dir /content/prepared_data

1000
3406
3401
Overall number of references: 3401


In [ ]:
with open('/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_en_en.hyp', 'r') as f:
  sentences = [i.strip() for i in f.readlines()]

lt = list()
st = set()
for i in sentences:
  if i not in st:
    lt.append(i)
    st.add(i)

with open('/content/model_prediction_en_en.hyp', 'w') as f:
  for i in lt:
    f.write(i+'\n')

In [ ]:
! easse evaluate \
--test_set custom \
--metrics sari \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction_en_en.hyp -q

[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
{'sari': 21.231, 'quality_estimation': {'Compression ratio': 0.935, 'Sentence splits': 1.034, 'Levenshtein similarity': 0.957, 'Exact copies': 0.419, 'Additions proportion': 0.019, 'Deletions proportion': 0.084, 'Lexical complexity score': 8.294}}


### BLEU

In [ ]:
# make lists of src, dst
src_column, trg_column = "en_en", "dst"
data_dict = OrderedDict()
for ind, row in data_test.iterrows():
        source_sentence = row[src_column]
        reference_sentence = row[trg_column]
        if data_dict.get(source_sentence) is None:
            data_dict[source_sentence] = []
        data_dict[source_sentence].append(reference_sentence)
max_num_reference_sentences = max([len(x) for x in data_dict.values()])

src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
for i, j in enumerate(src):
   if len(dst[i]) < max_num_reference_sentences:
    dst[i] = dst[i] + [dst[i][0]]*(max_num_reference_sentences - len(dst[i]))

refs = []
for j in range(len(dst[1])):
  ref = []
  for i in dst:
    ref.append(i[j])
  refs.append(ref)
  
corpus_bleu(src, refs)

41.05267985962619

### Flesh-Kincaid Grade level

In [ ]:
textstat.set_lang('en')

MEAN

In [ ]:
a = data_test['src'].values
en_en = sum(textstat.flesch_kincaid_grade(i) for i in a)/len(a)
en_en

11.071814445096855

In [ ]:
a = data_test['en_en'].values
en_en = sum(textstat.flesch_kincaid_grade(i) for i in a)/len(a)
en_en

11.808367586611897

WHOLE TEXT

In [ ]:
a = ' '.join(list(data_test['src'].values))
en_en = textstat.flesch_kincaid_grade(a)
en_en

10.7

In [ ]:
a = ' '.join(list(data_test['en_en'].values))
en_en = textstat.flesch_kincaid_grade(a)
en_en

11.1

In [ ]:
a = ' '.join(list(data_test['en_en'].values))
en_en = textstat.avg_sentence_length(a)
en_en

20.0

In [ ]:
a = ' '.join(list(data_test['en_en'].values))
en_en = textstat.avg_syllables_per_word(a)
en_en

1.6

In [ ]:
with open('/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_en_en.hyp', 'r') as f:
  sentences = [i.strip() for i in f.readlines()]

lt = list()
st = set()
for i in sentences:
  if i not in st:
    lt.append(i)
    st.add(i)

with open('/content/model_prediction_en_en.hyp', 'w') as f:
  for i in lt:
    f.write(i+'\n')

In [ ]:
! easse evaluate \
--test_set custom \
--metrics fkgl \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction_en_en.hyp -q


{'fkgl': 11.159, 'quality_estimation': {'Compression ratio': 0.929, 'Sentence splits': 1.042, 'Levenshtein similarity': 0.156, 'Exact copies': 0.0, 'Additions proportion': 0.869, 'Deletions proportion': 0.771, 'Lexical complexity score': 8.294}}


### Cosine Similarity

In [ ]:
from transformers import RobertaTokenizer, RobertaModel, AutoConfig, AutoTokenizer, AutoModelForMaskedLM
device = "cuda" if torch.cuda.is_available() else "cpu"
config = AutoConfig.from_pretrained("roberta-base") # "roberta-base" 'xlm-mlm-100-1280' 'xlm-roberta-base' 'bert-base-multilingual-cased'
config.output_hidden_states = True

tok = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModelForMaskedLM.from_pretrained("roberta-base", config=config)
model.to(device)

In [ ]:
calc_cos_sim(data_test, model, tok, 'src', 'en_en', 'cos_sim_en_en')
data_test.cos_sim_en_en.mean()

0.9784409024862598

### Grammar

In [ ]:
tool = language_tool_python.LanguageTool('en')

In [ ]:
errors = get_mistakes_summary(data_test, 'en_en')
errors

{'CASING': 38,
 'COLLOCATIONS': 2,
 'GRAMMAR': 23,
 'MISC': 1,
 'PUNCTUATION': 183,
 'REDUNDANCY': 28,
 'STYLE': 3,
 'TYPOGRAPHY': 8081,
 'TYPOS': 10}

 ## Evaluation of English model with filtering

### SARI

In [ ]:
with open('/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_en_en_filtered.hyp', 'r') as f:
  sentences = [i.strip() for i in f.readlines()]

lt = list()
st = set()
for i in sentences:
  if i not in st:
    lt.append(i)
    st.add(i)

with open('/content/model_prediction_en_en_filtered.hyp', 'w') as f:
  for i in lt:
    f.write(i+'\n')

In [ ]:
! easse evaluate \
--test_set custom \
--metrics sari \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction_en_en_filtered.hyp -q

{'sari': 34.596, 'quality_estimation': {'Compression ratio': 0.597, 'Sentence splits': 0.987, 'Levenshtein similarity': 0.727, 'Exact copies': 0.015, 'Additions proportion': 0.009, 'Deletions proportion': 0.414, 'Lexical complexity score': 8.217}}


### BLEU

In [ ]:
# make lists of src, dst
src_column, trg_column = "en_en_filtered", "dst"
data_dict = OrderedDict()
for ind, row in data_test.iterrows():
        source_sentence = row[src_column]
        reference_sentence = row[trg_column]
        if data_dict.get(source_sentence) is None:
            data_dict[source_sentence] = []
        data_dict[source_sentence].append(reference_sentence)
max_num_reference_sentences = max([len(x) for x in data_dict.values()])

src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
for i, j in enumerate(src):
   if len(dst[i]) < max_num_reference_sentences:
    dst[i] = dst[i] + [dst[i][0]]*(max_num_reference_sentences - len(dst[i]))

refs = []
for j in range(len(dst[1])):
  ref = []
  for i in dst:
    ref.append(i[j])
  refs.append(ref)
  
corpus_bleu(src, refs)

40.97846360104546


### FKLG

MEAN

In [ ]:
a = data_test['en_en_filtered'].values
en_en = sum(textstat.flesch_kincaid_grade(i) for i in a)/len(a)
en_en


9.058573106283028

WHOLE DATA

In [ ]:
a = ' '.join(list(data_test['src'].values))
en_en = textstat.flesch_kincaid_grade(a)
en_en

10.7

In [ ]:
a = ' '.join(list(data_test['src'].values))
en_en = textstat.avg_sentence_length(a)
en_en

22.1

In [ ]:
a = ' '.join(list(data_test['src'].values))
en_en = textstat.avg_syllables_per_word(a)
en_en

1.5

In [ ]:
a = ' '.join(list(data_test['dst'].values))
en_en = textstat.flesch_kincaid_grade(a)
en_en

8.6

In [ ]:
a = ' '.join(list(data_test['dst'].values))
en_en = textstat.avg_sentence_length(a)
en_en

16.6

In [ ]:
a = ' '.join(list(data_test['dst'].values))
en_en = textstat.avg_syllables_per_word(a)
en_en

1.5

In [ ]:
a = ' '.join(list(data_test['en_en_filtered'].values))
en_en = textstat.flesch_kincaid_grade(a)
en_en

8.4

In [ ]:
a = ' '.join(list(data_test['en_en_filtered'].values))
en_en = textstat.avg_sentence_length(a)
en_en

13.2

In [ ]:
a = ' '.join(list(data_test['en_en_filtered'].values))
en_en = textstat.avg_syllables_per_word(a)
en_en

1.6

In [ ]:
with open('/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_en_en_filtered.hyp', 'r') as f:
  sentences = [i.strip() for i in f.readlines()]

lt = list()
st = set()
for i in sentences:
  if i not in st:
    lt.append(i)
    st.add(i)

with open('/content/model_prediction_en_en_filtered.hyp', 'w') as f:
  for i in lt:
    f.write(i+'\n')


In [ ]:
! easse evaluate \
--test_set custom \
--metrics fkgl \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction_en_en_filtered.hyp -q


{'fkgl': 8.521, 'quality_estimation': {'Compression ratio': 0.591, 'Sentence splits': 0.99, 'Levenshtein similarity': 0.133, 'Exact copies': 0.0, 'Additions proportion': 0.635, 'Deletions proportion': 0.911, 'Lexical complexity score': 8.217}}


COSINE SIMILARITY

In [ ]:
from transformers import RobertaTokenizer, RobertaModel, AutoConfig, AutoTokenizer, AutoModelForMaskedLM
device = "cuda" if torch.cuda.is_available() else "cpu"
config = AutoConfig.from_pretrained("roberta-base") # "roberta-base" 'xlm-mlm-100-1280' 'xlm-roberta-base' 'bert-base-multilingual-cased'
config.output_hidden_states = True

tok = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModelForMaskedLM.from_pretrained("roberta-base", config=config)
model.to(device)

In [ ]:
calc_cos_sim(data_test, model, tok, 'src', 'en_en_filtered', 'cos_sim_en_en_filtered')
data_test.cos_sim_en_en_filtered.mean()

0.9042795725763981

### Grammar

In [ ]:
errors = get_mistakes_summary(data_test, 'en_en_filtered')
errors

{'CASING': 14,
 'GRAMMAR': 43,
 'PUNCTUATION': 29,
 'REDUNDANCY': 10,
 'TYPOGRAPHY': 5028,
 'TYPOS': 15}

## Evaluation of Russian model trained on data without filtering 

### SARI

In [ ]:
! rm -r /content/prepared_data
! mkdir /content/prepared_data

In [ ]:
! python /content/sent_simplification/refs_to_easse_format.py \
--input_path /content/drive/MyDrive/MT_sentence_simpl/wiki_test_dev_eng.csv \
--output_dataset_name test_ref_data \
--src_column "INPUT:source" \
--trg_column "OUTPUT:output" \
--output_dir /content/prepared_data

1000
3406
3406
Overall number of references: 3406


In [ ]:
with open('/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_ru_ru.hyp', 'r') as f:
  sentences = [i.strip() for i in f.readlines()]

lt = list()
st = set()
for i in sentences:
  if i not in st:
    lt.append(i)
    st.add(i)

with open('/content/model_prediction__ru_ru.hyp', 'w') as f:
  for i in lt:
    f.write(i+'\n')

In [ ]:
! easse evaluate \
--test_set custom \
--metrics sari \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction__ru_ru.hyp -q

{'sari': 16.499, 'quality_estimation': {'Compression ratio': 0.968, 'Sentence splits': 1.003, 'Levenshtein similarity': 0.979, 'Exact copies': 0.68, 'Additions proportion': 0.011, 'Deletions proportion': 0.045, 'Lexical complexity score': 10.675}}


### BLEU

In [ ]:
# make lists of src, dst
src_column, trg_column = "ru_ru", "OUTPUT:output"
data_dict = OrderedDict()
for ind, row in data_test.iterrows():
        source_sentence = row[src_column]
        reference_sentence = row[trg_column]
        if data_dict.get(source_sentence) is None:
            data_dict[source_sentence] = []
        data_dict[source_sentence].append(reference_sentence)
max_num_reference_sentences = max([len(x) for x in data_dict.values()])

src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
for i, j in enumerate(src):
   if len(dst[i]) < max_num_reference_sentences:
    dst[i] = dst[i] + [dst[i][0]]*(max_num_reference_sentences - len(dst[i]))

refs = []
for j in range(len(dst[1])):
  ref = []
  for i in dst:
    ref.append(i[j])
  refs.append(ref)
  
corpus_bleu(src, refs)

36.70629761889349

In [ ]:
refs = []
for i, j in data_dict.items():
  refs.append(j)

nltk.translate.bleu_score.corpus_bleu(refs, src)

0.7189872220505059

### Flesh-Kincaid Grade level

In [ ]:
calc_readability_metrics(' '.join(list(data_test['INPUT:source'].values)))

{'avg_slen': 16.790763968072977,
 'avg_syl': 2.8487487691419644,
 'c_share': 16.036806899595938,
 'chars': 473430,
 'index_SMOG_rus': 14.557852197020122,
 'index_ari_rus': 15.509208294776158,
 'index_cl_rus': 14.345376048351504,
 'index_dc_rus': 13.436195971860881,
 'index_fk_rus': 15.291695463663103,
 'letters': 393678,
 'n_complex_words': 9446,
 'n_sentences': 3508,
 'n_simple_words': 46179,
 'n_syllabes': 167797,
 'n_words': 58902,
 'spaces': 60292,
 'wsyllabes': {'1': 9867,
  '10': 35,
  '11': 6,
  '12': 4,
  '2': 12788,
  '3': 13723,
  '4': 9801,
  '5': 6023,
  '6': 2491,
  '7': 677,
  '8': 173,
  '9': 37}}

In [ ]:
calc_readability_metrics(' '.join(list(data_test['OUTPUT:output'].values)))

{'avg_slen': 13.503585397653195,
 'avg_syl': 2.6891790774578195,
 'c_share': 12.112288493567307,
 'chars': 317164,
 'index_SMOG_rus': 11.356979245625993,
 'index_ari_rus': 11.940864878774391,
 'index_cl_rus': 11.512936107557508,
 'index_dc_rus': 10.372462062008477,
 'index_fk_rus': 11.98048305635475,
 'letters': 259382,
 'n_complex_words': 5018,
 'n_sentences': 3068,
 'n_simple_words': 34238,
 'n_syllabes': 111410,
 'n_words': 41429,
 'spaces': 43375,
 'wsyllabes': {'1': 7690,
  '10': 11,
  '11': 1,
  '12': 1,
  '14': 1,
  '15': 1,
  '2': 10187,
  '3': 9546,
  '4': 6815,
  '5': 3294,
  '6': 1274,
  '7': 336,
  '8': 71,
  '9': 28}}

In [ ]:
calc_readability_metrics(' '.join(list(data_test.ru_ru.values)))

{'avg_slen': 16.305825242718445,
 'avg_syl': 2.8488520743218393,
 'c_share': 15.920354447226941,
 'chars': 458434,
 'index_SMOG_rus': 14.294811353101062,
 'index_ari_rus': 15.356937930462564,
 'index_cl_rus': 14.269110379489696,
 'index_dc_rus': 13.239525946131408,
 'index_fk_rus': 15.13895188255458,
 'letters': 381506,
 'n_complex_words': 9091,
 'n_sentences': 3502,
 'n_simple_words': 44882,
 'n_syllabes': 162678,
 'n_words': 57103,
 'spaces': 58381,
 'wsyllabes': {'1': 9581,
  '10': 35,
  '11': 6,
  '12': 4,
  '2': 12369,
  '3': 13339,
  '4': 9593,
  '5': 5826,
  '6': 2384,
  '7': 648,
  '8': 156,
  '9': 32}}

COSINE SIMILARITY

In [ ]:
from transformers import RobertaTokenizer, RobertaModel, AutoConfig, AutoTokenizer, AutoModelForMaskedLM
device = "cuda" if torch.cuda.is_available() else "cpu"
config = AutoConfig.from_pretrained("DeepPavlov/rubert-base-cased") # "roberta-base" 'xlm-mlm-100-1280' 'xlm-roberta-base' 'bert-base-multilingual-cased'
config.output_hidden_states = True

tok = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = AutoModelForMaskedLM.from_pretrained("DeepPavlov/rubert-base-cased", config=config)
model.to(device)

Some weights of BertForMaskedLM were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=T

In [ ]:
calc_cos_sim(data_test, model, tok, 'INPUT:source', 'ru_ru', 'cos_sim_ru_ru')
data_test.cos_sim_ru_ru.mean()

0.9955931413422174

### Grammar

In [ ]:
tool = language_tool_python.LanguageTool('ru')

Unzipping /tmp/tmpjs52uptj.zip to /root/.cache/language_tool_python.
Downloaded https://www.languagetool.org/download/LanguageTool-5.2.zip to /root/.cache/language_tool_python.


In [ ]:
errors = get_mistakes_summary(data_test, 'ru_ru')
errors

{'CASING': 8,
 'GRAMMAR': 22,
 'LOGIC': 27,
 'PUNCTUATION': 20,
 'STYLE': 14,
 'TYPOGRAPHY': 393,
 'TYPOS': 800}

## Evaluation of Russian model without filtering pretrained on English data

### SARI

In [ ]:
# ! rm -r /content/prepared_data
# ! mkdir /content/prepared_data

In [ ]:
# ! python /content/sent_simplification/refs_to_easse_format.py \
# --input_path /content/drive/MyDrive/MT_sentence_simpl/wiki_test_dev_eng.csv \
# --output_dataset_name test_ref_data \
# --src_column "INPUT:source" \
# --trg_column "OUTPUT:output" \
# --output_dir /content/prepared_data

1000
3406
3406
Overall number of references: 3406


In [ ]:
with open('/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_en_ru_ru.hyp', 'r') as f:
  sentences = [i.strip() for i in f.readlines()]

lt = list()
st = set()
for i in sentences:
  if i not in st:
    lt.append(i)
    st.add(i)

with open('/content/model_prediction_en_ru_ru.hyp', 'w') as f:
  for i in lt:
    f.write(i+'\n')

In [ ]:
! easse evaluate \
--test_set custom \
--metrics sari \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction_en_ru_ru.hyp -q

{'sari': 16.939, 'quality_estimation': {'Compression ratio': 0.963, 'Sentence splits': 1.002, 'Levenshtein similarity': 0.975, 'Exact copies': 0.652, 'Additions proportion': 0.013, 'Deletions proportion': 0.051, 'Lexical complexity score': 10.682}}


### BLEU

In [ ]:
# make lists of src, dst
# src_column, trg_column = "en_ru_ru", "OUTPUT:output"
# data_dict = OrderedDict()
# for ind, row in data_test.iterrows():
#         source_sentence = row[src_column]
#         reference_sentence = row[trg_column]
#         if data_dict.get(source_sentence) is None:
#             data_dict[source_sentence] = []
#         data_dict[source_sentence].append(reference_sentence)
# max_num_reference_sentences = max([len(x) for x in data_dict.values()])
# src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
# corpus_averaged_sentence_bleu(src, dst)

34.749903130866464

In [ ]:
# make lists of src, dst
src_column, trg_column = "en_ru_ru", "OUTPUT:output"
data_dict = OrderedDict()
for ind, row in data_test.iterrows():
        source_sentence = row[src_column]
        reference_sentence = row[trg_column]
        if data_dict.get(source_sentence) is None:
            data_dict[source_sentence] = []
        data_dict[source_sentence].append(reference_sentence)
max_num_reference_sentences = max([len(x) for x in data_dict.values()])

src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
for i, j in enumerate(src):
   if len(dst[i]) < max_num_reference_sentences:
    dst[i] = dst[i] + [dst[i][0]]*(max_num_reference_sentences - len(dst[i]))

refs = []
for j in range(len(dst[1])):
  ref = []
  for i in dst:
    ref.append(i[j])
  refs.append(ref)
  
corpus_bleu(src, refs)

36.94343271922458

In [ ]:
refs = []
for i, j in data_dict.items():
  refs.append(j)

nltk.translate.bleu_score.corpus_bleu(refs, src)

0.7200647332702366

FKGL

In [ ]:
calc_readability_metrics(' '.join(list(data_test.en_ru_ru.values)))

{'avg_slen': 16.21505989731888,
 'avg_syl': 2.8492700087950746,
 'c_share': 15.980650835532103,
 'chars': 456511,
 'index_SMOG_rus': 14.281984238205567,
 'index_ari_rus': 15.354965286246319,
 'index_cl_rus': 14.277730870712404,
 'index_dc_rus': 13.248030613181776,
 'index_fk_rus': 15.116023172237462,
 'letters': 380029,
 'n_complex_words': 9085,
 'n_sentences': 3506,
 'n_simple_words': 44651,
 'n_syllabes': 161981,
 'n_words': 56850,
 'spaces': 58090,
 'wsyllabes': {'1': 9523,
  '10': 35,
  '11': 6,
  '12': 4,
  '2': 12377,
  '3': 13254,
  '4': 9497,
  '5': 5829,
  '6': 2373,
  '7': 636,
  '8': 163,
  '9': 39}}

COSINE SIMILARITY

In [ ]:
calc_cos_sim(data_test, model, tok, 'INPUT:source', 'ru_ru', 'cos_sim_en_ru_ru')
data_test.cos_sim_en_ru_ru.mean()

0.9955931413422174

### Grammar

In [ ]:
errors = get_mistakes_summary(data_test, 'en_ru_ru')
errors

{'CASING': 17,
 'GRAMMAR': 20,
 'LOGIC': 27,
 'PUNCTUATION': 21,
 'STYLE': 14,
 'TYPOGRAPHY': 398,
 'TYPOS': 800}

# Evaluation of Russian model trained on filtered data

### SARI

In [ ]:
# ! rm -r /content/prepared_data
# ! mkdir /content/prepared_data

! python /content/sent_simplification/refs_to_easse_format.py \
--input_path /content/drive/MyDrive/MT_sentence_simpl/wiki_test_dev_eng.csv \
--output_dataset_name test_ref_data \
--src_column "INPUT:source" \
--trg_column "OUTPUT:output" \
--output_dir /content/prepared_data

1000
3406
3406
Overall number of references: 3406


In [ ]:
with open('/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_ru_ru_filtered.hyp', 'r') as f:
  sentences = [i.strip() for i in f.readlines()]

lt = list()
st = set()
for i in sentences:
  if i not in st:
    lt.append(i)
    st.add(i)

with open('/content/model_prediction_ru_ru_filtered.hyp', 'w') as f:
  for i in lt:
    f.write(i+'\n')

In [ ]:
! easse evaluate \
--test_set custom \
--metrics sari \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction_ru_ru_filtered.hyp -q

{'sari': 32.31, 'quality_estimation': {'Compression ratio': 0.661, 'Sentence splits': 0.988, 'Levenshtein similarity': 0.778, 'Exact copies': 0.014, 'Additions proportion': 0.014, 'Deletions proportion': 0.361, 'Lexical complexity score': 10.721}}


### BLEU

In [ ]:
# make lists of src, dst
# src_column, trg_column = "ru_ru_filtered", "OUTPUT:output"
# data_dict = OrderedDict()
# for ind, row in data_test.iterrows():
#         source_sentence = row[src_column]
#         reference_sentence = row[trg_column]
#         if data_dict.get(source_sentence) is None:
#             data_dict[source_sentence] = []
#         data_dict[source_sentence].append(reference_sentence)
# max_num_reference_sentences = max([len(x) for x in data_dict.values()])
# src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
# corpus_averaged_sentence_bleu(src, dst)

53.16967153331756

In [ ]:
# make lists of src, dst
src_column, trg_column = "ru_ru_filtered", "OUTPUT:output"
data_dict = OrderedDict()
for ind, row in data_test.iterrows():
        source_sentence = row[src_column]
        reference_sentence = row[trg_column]
        if data_dict.get(source_sentence) is None:
            data_dict[source_sentence] = []
        data_dict[source_sentence].append(reference_sentence)
max_num_reference_sentences = max([len(x) for x in data_dict.values()])

src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
for i, j in enumerate(src):
   if len(dst[i]) < max_num_reference_sentences:
    dst[i] = dst[i] + [dst[i][0]]*(max_num_reference_sentences - len(dst[i]))

refs = []
for j in range(len(dst[1])):
  ref = []
  for i in dst:
    ref.append(i[j])
  refs.append(ref)
  
corpus_bleu(src, refs)

37.2531944122807

In [ ]:
refs = []
for i, j in data_dict.items():
  refs.append(j)

nltk.translate.bleu_score.corpus_bleu(refs, src)

0.7725918655216105

### FKLG

In [ ]:
calc_readability_metrics(' '.join(list(data_test.ru_ru_filtered.values)))

{'avg_slen': 11.11542957334892,
 'avg_syl': 2.8846123511317927,
 'c_share': 15.910823671688092,
 'chars': 310421,
 'index_SMOG_rus': 11.807592421245358,
 'index_ari_rus': 14.509345250891315,
 'index_cl_rus': 13.801266924310546,
 'index_dc_rus': 11.817286940296082,
 'index_fk_rus': 13.996201990396415,
 'letters': 257822,
 'n_complex_words': 6052,
 'n_sentences': 3422,
 'n_simple_words': 30130,
 'n_syllabes': 109722,
 'n_words': 38037,
 'spaces': 38974,
 'wsyllabes': {'1': 5850,
  '10': 18,
  '11': 6,
  '12': 1,
  '2': 8631,
  '3': 9238,
  '4': 6411,
  '5': 3882,
  '6': 1585,
  '7': 428,
  '8': 110,
  '9': 22}}

COSINE SIMILARITY

In [ ]:
calc_cos_sim(data_test, model, tok, 'INPUT:source', 'ru_ru_filtered', 'cos_sim_ru_ru_filtered')
data_test.cos_sim_ru_ru_filtered.mean()

0.9557879034449476

### Grammar

In [ ]:
errors = get_mistakes_summary(data_test, 'ru_ru_filtered')
errors

{'CASING': 32,
 'GRAMMAR': 31,
 'LOGIC': 27,
 'PUNCTUATION': 22,
 'STYLE': 2,
 'TYPOGRAPHY': 197,
 'TYPOS': 571}

# Evaluation of Russian model trained on filtered data and pretrained on english data

### SARI

In [ ]:
with open('/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_en_ru_ru_filtered.hyp', 'r') as f:
  sentences = [i.strip() for i in f.readlines()]

lt = list()
st = set()
for i in sentences:
  if i not in st:
    lt.append(i)
    st.add(i)

with open('/content/model_prediction_en_ru_ru_filtered.hyp', 'w') as f:
  for i in lt:
    f.write(i+'\n')

In [ ]:
! easse evaluate \
--test_set custom \
--metrics sari \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction_en_ru_ru_filtered.hyp -q

{'sari': 32.234, 'quality_estimation': {'Compression ratio': 0.658, 'Sentence splits': 0.988, 'Levenshtein similarity': 0.775, 'Exact copies': 0.018, 'Additions proportion': 0.017, 'Deletions proportion': 0.367, 'Lexical complexity score': 10.727}}


### BLEU

In [ ]:
# make lists of src, dst
# src_column, trg_column = "en_ru_ru_filtered", "OUTPUT:output"
# data_dict = OrderedDict()
# for ind, row in data_test.iterrows():
#         source_sentence = row[src_column]
#         reference_sentence = row[trg_column]
#         if data_dict.get(source_sentence) is None:
#             data_dict[source_sentence] = []
#         data_dict[source_sentence].append(reference_sentence)
# max_num_reference_sentences = max([len(x) for x in data_dict.values()])
# src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
# corpus_averaged_sentence_bleu(src, dst)

53.16967153331756

In [ ]:
# make lists of src, dst
src_column, trg_column = "en_ru_ru_filtered", "OUTPUT:output"
data_dict = OrderedDict()
for ind, row in data_test.iterrows():
        source_sentence = row[src_column]
        reference_sentence = row[trg_column]
        if data_dict.get(source_sentence) is None:
            data_dict[source_sentence] = []
        data_dict[source_sentence].append(reference_sentence)
max_num_reference_sentences = max([len(x) for x in data_dict.values()])

src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
for i, j in enumerate(src):
   if len(dst[i]) < max_num_reference_sentences:
    dst[i] = dst[i] + [dst[i][0]]*(max_num_reference_sentences - len(dst[i]))

refs = []
for j in range(len(dst[1])):
  ref = []
  for i in dst:
    ref.append(i[j])
  refs.append(ref)
  
corpus_bleu(src, refs)

36.79295860256343

In [ ]:
refs = []
for i, j in data_dict.items():
  refs.append(j)

nltk.translate.bleu_score.corpus_bleu(refs, src)

0.7746458621440553

### FKLG

In [ ]:
calc_readability_metrics(' '.join(list(data_test.en_ru_ru_filtered.values)))

{'avg_slen': 11.183596030356101,
 'avg_syl': 2.887798512331985,
 'c_share': 16.01983557353517,
 'chars': 312398,
 'index_SMOG_rus': 11.883922121789904,
 'index_ari_rus': 14.494590752285468,
 'index_cl_rus': 13.790696855017622,
 'index_dc_rus': 11.89607095287863,
 'index_fk_rus': 14.06312241276742,
 'letters': 259499,
 'n_complex_words': 6138,
 'n_sentences': 3426,
 'n_simple_words': 30281,
 'n_syllabes': 110646,
 'n_words': 38315,
 'spaces': 39266,
 'wsyllabes': {'1': 5932,
  '10': 25,
  '11': 6,
  '12': 1,
  '2': 8604,
  '3': 9235,
  '4': 6510,
  '5': 3952,
  '6': 1564,
  '7': 450,
  '8': 121,
  '9': 19}}

In [ ]:
calc_cos_sim(data_test, model, tok, 'INPUT:source', 'en_ru_ru_filtered', 'cos_sim_en_ru_ru_filtered')
data_test.cos_sim_en_ru_ru_filtered.mean()

0.9578453072041045

### Grammar

In [ ]:
errors = get_mistakes_summary(data_test, 'en_ru_ru_filtered')
errors

{'CASING': 35,
 'GRAMMAR': 33,
 'LOGIC': 27,
 'PUNCTUATION': 11,
 'STYLE': 1,
 'TYPOGRAPHY': 218,
 'TYPOS': 578}

# Evaluation of Russian model trained on filtered data and pretrained on paraphraser

### SARI

In [ ]:
with open('/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_paraphraser_ru2_12epochs.hyp', 'r') as f:
  sentences = [i.strip() for i in f.readlines()]

lt = list()
st = set()
for i in sentences:
  if i not in st:
    lt.append(i)
    st.add(i)

with open('/content/model_prediction_paraphraser_ru2_12epochs.hyp', 'w') as f:
  for i in lt:
    f.write(i+'\n')

In [ ]:
! easse evaluate \
--test_set custom \
--metrics sari \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction_paraphraser_ru2_12epochs.hyp -q

{'sari': 31.812, 'quality_estimation': {'Compression ratio': 0.694, 'Sentence splits': 0.987, 'Levenshtein similarity': 0.798, 'Exact copies': 0.051, 'Additions proportion': 0.014, 'Deletions proportion': 0.332, 'Lexical complexity score': 10.707}}


### BLEU

In [ ]:
# make lists of src, dst
# src_column, trg_column = "paraphraser_ru", "OUTPUT:output"
# data_dict = OrderedDict()
# for ind, row in data_test.iterrows():
#         source_sentence = row[src_column]
#         reference_sentence = row[trg_column]
#         if data_dict.get(source_sentence) is None:
#             data_dict[source_sentence] = []
#         data_dict[source_sentence].append(reference_sentence)
# max_num_reference_sentences = max([len(x) for x in data_dict.values()])
# src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
# corpus_averaged_sentence_bleu(src, dst)

53.16967153331756

In [ ]:
# make lists of src, dst
src_column, trg_column = "paraphraser_ru", "OUTPUT:output"
data_dict = OrderedDict()
for ind, row in data_test.iterrows():
        source_sentence = row[src_column]
        reference_sentence = row[trg_column]
        if data_dict.get(source_sentence) is None:
            data_dict[source_sentence] = []
        data_dict[source_sentence].append(reference_sentence)
max_num_reference_sentences = max([len(x) for x in data_dict.values()])

src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
for i, j in enumerate(src):
   if len(dst[i]) < max_num_reference_sentences:
    dst[i] = dst[i] + [dst[i][0]]*(max_num_reference_sentences - len(dst[i]))

refs = []
for j in range(len(dst[1])):
  ref = []
  for i in dst:
    ref.append(i[j])
  refs.append(ref)
  
corpus_bleu(src, refs)

37.98414836939204

In [ ]:
refs = []
for i, j in data_dict.items():
  refs.append(j)

nltk.translate.bleu_score.corpus_bleu(refs, src)

0.7706207829780775

### FKLG

In [ ]:
calc_readability_metrics(' '.join(list(data_test.paraphraser_ru.values)))

{'avg_slen': 11.698250728862973,
 'avg_syl': 2.880299065420561,
 'c_share': 15.937694704049845,
 'chars': 325728,
 'index_SMOG_rus': 12.122081494236964,
 'index_ari_rus': 14.536493535053538,
 'index_cl_rus': 13.83836137071652,
 'index_dc_rus': 11.991229925615107,
 'index_fk_rus': 14.12029046075039,
 'letters': 271101,
 'n_complex_words': 6395,
 'n_sentences': 3430,
 'n_simple_words': 31695,
 'n_syllabes': 115572,
 'n_words': 40125,
 'spaces': 41033,
 'wsyllabes': {'1': 6297,
  '10': 23,
  '11': 6,
  '12': 1,
  '2': 8982,
  '3': 9561,
  '4': 6855,
  '5': 4047,
  '6': 1717,
  '7': 464,
  '8': 118,
  '9': 19}}

In [ ]:
calc_cos_sim(data_test, model, tok, 'INPUT:source', 'paraphraser_ru', 'cos_sim_paraphraser_ru')
data_test.cos_sim_paraphraser_ru.mean()

0.9564811277410527

### Grammar

In [ ]:
errors = get_mistakes_summary(data_test, 'paraphraser_ru')
errors

{'CASING': 22,
 'GRAMMAR': 39,
 'LOGIC': 27,
 'PUNCTUATION': 29,
 'TYPOGRAPHY': 183,
 'TYPOS': 605}

# Evaluation of Russian model trained on Paraphraser

### SARI

In [ ]:
with open('/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_paraphraser.hyp', 'r') as f:
  sentences = [i.strip()+'.' for i in f.readlines()]

lt = list()
st = set()
for i in sentences:
  if i not in st:
    lt.append(i)
    st.add(i)

with open('/content/model_prediction_paraphraser.hyp', 'w') as f:
  for i in lt:
    f.write(i+'\n')

In [ ]:
! easse evaluate \
--test_set custom \
--metrics sari \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction_paraphraser.hyp -q

{'sari': 35.149, 'quality_estimation': {'Compression ratio': 0.624, 'Sentence splits': 0.985, 'Levenshtein similarity': 0.711, 'Exact copies': 0.026, 'Additions proportion': 0.053, 'Deletions proportion': 0.441, 'Lexical complexity score': 10.732}}


### BLEU

In [ ]:
# make lists of src, dst
# src_column, trg_column = "paraphraser", "OUTPUT:output"
# data_dict = OrderedDict()
# for ind, row in data_test.iterrows():
#         source_sentence = row[src_column]
#         reference_sentence = row[trg_column]
#         if data_dict.get(source_sentence) is None:
#             data_dict[source_sentence] = []
#         data_dict[source_sentence].append(reference_sentence)
# max_num_reference_sentences = max([len(x) for x in data_dict.values()])
# src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
# corpus_averaged_sentence_bleu(src, dst)

In [ ]:
# make lists of src, dst
src_column, trg_column = "paraphraser", "OUTPUT:output"
data_dict = OrderedDict()
for ind, row in data_test.iterrows():
        source_sentence = row[src_column]
        reference_sentence = row[trg_column]
        if data_dict.get(source_sentence) is None:
            data_dict[source_sentence] = []
        data_dict[source_sentence].append(reference_sentence)
max_num_reference_sentences = max([len(x) for x in data_dict.values()])

src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
for i, j in enumerate(src):
   if len(dst[i]) < max_num_reference_sentences:
    dst[i] = dst[i] + [dst[i][0]]*(max_num_reference_sentences - len(dst[i]))

refs = []
for j in range(len(dst[1])):
  ref = []
  for i in dst:
    ref.append(i[j])
  refs.append(ref)
  
corpus_bleu(src, refs)

30.314500674726755

In [ ]:
refs = []
for i, j in data_dict.items():
  refs.append(j)

nltk.translate.bleu_score.corpus_bleu(refs, src)

0.7206319953649845

### FKLG

In [ ]:
calc_readability_metrics(' '.join(list(data_test.paraphraser.values)))

{'avg_slen': 10.044113350861817,
 'avg_syl': 2.8966580378697535,
 'c_share': 16.067013757598673,
 'chars': 281270,
 'index_SMOG_rus': 11.28135952373477,
 'index_ari_rus': 14.362525477532138,
 'index_cl_rus': 13.600441232075852,
 'index_dc_rus': 11.611034538979744,
 'index_fk_rus': 13.826572183324558,
 'letters': 233885,
 'n_complex_words': 5524,
 'n_sentences': 3423,
 'n_simple_words': 27013,
 'n_syllabes': 99590,
 'n_words': 34381,
 'spaces': 35374,
 'wsyllabes': {'1': 5184,
  '10': 18,
  '11': 1,
  '12': 5,
  '2': 7437,
  '3': 8443,
  '4': 5949,
  '5': 3489,
  '6': 1503,
  '7': 387,
  '8': 105,
  '9': 16}}

COSINE SIMILARITY

In [ ]:
calc_cos_sim(data_test, model, tok, 'INPUT:source', 'paraphraser', 'cos_sim_paraphraser')
data_test.cos_sim_paraphraser.mean()

0.9374921821709667

### Grammar

In [ ]:
errors = get_mistakes_summary(data_test, 'paraphraser')
errors

{'CASING': 29,
 'GRAMMAR': 29,
 'LOGIC': 27,
 'MISC': 9,
 'PUNCTUATION': 30,
 'STYLE': 1,
 'TYPOGRAPHY': 158,
 'TYPOS': 692}

# BONUS EN-RU MODEL

### SARI

In [ ]:
with open('/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_en_ru.hyp', 'r') as f:
  sentences = [i.strip() for i in f.readlines()]

lt = list()
st = set()
for i in sentences:
  if i not in st:
    lt.append(i)
    st.add(i)

with open('/content/model_prediction_en_ru.hyp', 'w') as f:
  for i in lt:
    f.write(i+'\n')

In [ ]:
! easse evaluate \
--test_set custom \
--metrics sari \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction_en_ru.hyp -q

{'sari': 37.865, 'quality_estimation': {'Compression ratio': 0.875, 'Sentence splits': 1.002, 'Levenshtein similarity': 0.665, 'Exact copies': 0.0, 'Additions proportion': 0.4, 'Deletions proportion': 0.497, 'Lexical complexity score': 10.398}}


### BLEU

In [ ]:
# make lists of src, dst
src_column, trg_column = "en_ru", "OUTPUT:output"
data_dict = OrderedDict()
for ind, row in data_test.iterrows():
        source_sentence = row[src_column]
        reference_sentence = row[trg_column]
        if data_dict.get(source_sentence) is None:
            data_dict[source_sentence] = []
        data_dict[source_sentence].append(reference_sentence)
max_num_reference_sentences = max([len(x) for x in data_dict.values()])

src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
for i, j in enumerate(src):
   if len(dst[i]) < max_num_reference_sentences:
    dst[i] = dst[i] + [dst[i][0]]*(max_num_reference_sentences - len(dst[i]))

refs = []
for j in range(len(dst[1])):
  ref = []
  for i in dst:
    ref.append(i[j])
  refs.append(ref)
  
corpus_bleu(src, refs) ###???

13.214882473186714

In [ ]:
refs = []
for i, j in data_dict.items():
  refs.append(j)

nltk.translate.bleu_score.corpus_bleu(refs, src)

0.5688826735511077

FKLG

In [ ]:
data_test.head()

In [ ]:
calc_readability_metrics(' '.join(list(data_test['INPUT:source'].values)))

{'avg_slen': 16.790763968072977,
 'avg_syl': 2.8487487691419644,
 'c_share': 16.036806899595938,
 'chars': 473430,
 'index_SMOG_rus': 14.557852197020122,
 'index_ari_rus': 15.509208294776158,
 'index_cl_rus': 14.345376048351504,
 'index_dc_rus': 13.436195971860881,
 'index_fk_rus': 15.291695463663103,
 'letters': 393678,
 'n_complex_words': 9446,
 'n_sentences': 3508,
 'n_simple_words': 46179,
 'n_syllabes': 167797,
 'n_words': 58902,
 'spaces': 60292,
 'wsyllabes': {'1': 9867,
  '10': 35,
  '11': 6,
  '12': 4,
  '2': 12788,
  '3': 13723,
  '4': 9801,
  '5': 6023,
  '6': 2491,
  '7': 677,
  '8': 173,
  '9': 37}}

In [ ]:
calc_readability_metrics(' '.join(list(data_test['OUTPUT:output'].values)))

{'avg_slen': 13.503585397653195,
 'avg_syl': 2.6891790774578195,
 'c_share': 12.112288493567307,
 'chars': 317164,
 'index_SMOG_rus': 11.356979245625993,
 'index_ari_rus': 11.940864878774391,
 'index_cl_rus': 11.512936107557508,
 'index_dc_rus': 10.372462062008477,
 'index_fk_rus': 11.98048305635475,
 'letters': 259382,
 'n_complex_words': 5018,
 'n_sentences': 3068,
 'n_simple_words': 34238,
 'n_syllabes': 111410,
 'n_words': 41429,
 'spaces': 43375,
 'wsyllabes': {'1': 7690,
  '10': 11,
  '11': 1,
  '12': 1,
  '14': 1,
  '15': 1,
  '2': 10187,
  '3': 9546,
  '4': 6815,
  '5': 3294,
  '6': 1274,
  '7': 336,
  '8': 71,
  '9': 28}}

In [ ]:
calc_readability_metrics(' '.join(list(data_test.en_ru.values)))

{'avg_slen': 14.71934838525293,
 'avg_syl': 2.7287148321457004,
 'c_share': 13.271071588062831,
 'chars': 411959,
 'index_SMOG_rus': 12.406806113663269,
 'index_ari_rus': 12.86566051041558,
 'index_cl_rus': 12.239918257188904,
 'index_dc_rus': 11.344013625784733,
 'index_fk_rus': 12.928503402979374,
 'letters': 327257,
 'n_complex_words': 6835,
 'n_sentences': 3499,
 'n_simple_words': 41701,
 'n_syllabes': 140537,
 'n_words': 51503,
 'spaces': 54374,
 'wsyllabes': {'1': 8966,
  '10': 4,
  '2': 12415,
  '3': 11739,
  '4': 8581,
  '5': 4441,
  '6': 1876,
  '7': 428,
  '8': 71,
  '9': 15}}

COSINE SIMILARITY

In [ ]:
calc_cos_sim(data_test, model, tok, 'INPUT:source', 'en_ru', 'cos_sim_en_ru')
data_test.cos_sim_en_ru.mean()

0.9811948919324265

### Grammar

In [ ]:
errors = get_mistakes_summary(data_test, 'en_ru')
errors

{'CASING': 57,
 'GRAMMAR': 163,
 'LOGIC': 12,
 'MISC': 131,
 'PUNCTUATION': 48,
 'STYLE': 27,
 'TYPOGRAPHY': 230,
 'TYPOS': 1073}

# Model trained 15 epochs on russian data

### SARI

In [ ]:
with open('/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction__ru_ru_filtered15.hyp', 'r') as f:
  sentences = [i.strip() for i in f.readlines()]

lt = list()
st = set()
for i in sentences:
  if i not in st:
    lt.append(i)
    st.add(i)

with open('/content/model_prediction__ru_ru_filtered15.hyp', 'w') as f:
  for i in lt:
    f.write(i+'\n')


In [ ]:
! easse evaluate \
--test_set custom \
--metrics sari \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction__ru_ru_filtered15.hyp -q


{'sari': 32.957, 'quality_estimation': {'Compression ratio': 0.635, 'Sentence splits': 0.987, 'Levenshtein similarity': 0.757, 'Exact copies': 0.008, 'Additions proportion': 0.019, 'Deletions proportion': 0.392, 'Lexical complexity score': 10.724}}


#### BLEU

In [ ]:
# make lists of src, dst
src_column, trg_column = "ru_15", "OUTPUT:output"
data_dict = OrderedDict()
for ind, row in data_test.iterrows():
        source_sentence = row[src_column]
        reference_sentence = row[trg_column]
        if data_dict.get(source_sentence) is None:
            data_dict[source_sentence] = []
        data_dict[source_sentence].append(reference_sentence)
max_num_reference_sentences = max([len(x) for x in data_dict.values()])

src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
for i, j in enumerate(src):
   if len(dst[i]) < max_num_reference_sentences:
    dst[i] = dst[i] + [dst[i][0]]*(max_num_reference_sentences - len(dst[i]))

refs = []
for j in range(len(dst[1])):
  ref = []
  for i in dst:
    ref.append(i[j])
  refs.append(ref)
  
corpus_bleu(src, refs)

35.99960438432429

In [ ]:
refs = []
for i, j in data_dict.items():
  refs.append(j)

nltk.translate.bleu_score.corpus_bleu(refs, src)

0.7713335235629974

#### FKGL

In [ ]:
calc_readability_metrics(' '.join(list(data_test.ru_15.values)))


{'avg_slen': 10.689252336448599,
 'avg_syl': 2.88672131147541,
 'c_share': 15.986338797814208,
 'chars': 298930,
 'index_SMOG_rus': 11.607319387523807,
 'index_ari_rus': 14.351192110974921,
 'index_cl_rus': 13.641803278688528,
 'index_dc_rus': 11.742624904243911,
 'index_fk_rus': 13.890624865941476,
 'letters': 247856,
 'n_complex_words': 5851,
 'n_sentences': 3424,
 'n_simple_words': 28980,
 'n_syllabes': 105654,
 'n_words': 36600,
 'spaces': 37553,
 'wsyllabes': {'1': 5650,
  '10': 20,
  '11': 6,
  '12': 1,
  '2': 8311,
  '3': 8884,
  '4': 6135,
  '5': 3733,
  '6': 1543,
  '7': 419,
  '8': 105,
  '9': 24}}

#### Cosine similarity

In [ ]:
calc_cos_sim(data_test, model, tok, 'INPUT:source', 'ru_15', 'cos_sim_ru_15')
data_test.cos_sim_ru_15.mean()

0.9548124609995224

##### Grammar

In [ ]:
errors = get_mistakes_summary(data_test, 'ru_15')
errors

{'CASING': 35,
 'GRAMMAR': 21,
 'LOGIC': 22,
 'PUNCTUATION': 18,
 'STYLE': 2,
 'TYPOGRAPHY': 210,
 'TYPOS': 554}

# Model trained 10 epochs on russian data and 10 on ParaPhraserPlus

### SARI

In [ ]:
with open('/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_ru22_par_filtered15.hyp', 'r') as f:
  sentences = [i.strip()+'.' for i in f.readlines()]

lt = list()
st = set()
for i in sentences:
  if i not in st:
    lt.append(i)
    st.add(i)

with open('/content/model_prediction_ru22_par_filtered15.hyp', 'w') as f:
  for i in lt:
    f.write(i+'\n')

In [ ]:
! easse evaluate \
--test_set custom \
--metrics sari \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction_ru22_par_filtered15.hyp -q


{'sari': 34.738, 'quality_estimation': {'Compression ratio': 0.632, 'Sentence splits': 0.986, 'Levenshtein similarity': 0.721, 'Exact copies': 0.039, 'Additions proportion': 0.045, 'Deletions proportion': 0.418, 'Lexical complexity score': 10.734}}


#### BLEU

In [ ]:
# make lists of src, dst
src_column, trg_column = "ru_paraphraser", "OUTPUT:output"
data_dict = OrderedDict()
for ind, row in data_test.iterrows():
        source_sentence = row[src_column]
        reference_sentence = row[trg_column]
        if data_dict.get(source_sentence) is None:
            data_dict[source_sentence] = []
        data_dict[source_sentence].append(reference_sentence)
max_num_reference_sentences = max([len(x) for x in data_dict.values()])

src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
for i, j in enumerate(src):
   if len(dst[i]) < max_num_reference_sentences:
    dst[i] = dst[i] + [dst[i][0]]*(max_num_reference_sentences - len(dst[i]))

refs = []
for j in range(len(dst[1])):
  ref = []
  for i in dst:
    ref.append(i[j])
  refs.append(ref)
  
corpus_bleu(src, refs)

33.980377072614985

In [ ]:
refs = []
for i, j in data_dict.items():
  refs.append(j)

nltk.translate.bleu_score.corpus_bleu(refs, src)

0.7433356957738314

#### FKGL

In [ ]:
calc_readability_metrics(' '.join(list(data_test.ru_paraphraser.values)))


{'avg_slen': 10.58732476635514,
 'avg_syl': 2.8722517999503463,
 'c_share': 15.588535488676175,
 'chars': 294631,
 'index_SMOG_rus': 11.408074678271678,
 'index_ari_rus': 14.150164447449505,
 'index_cl_rus': 13.458778240600267,
 'index_dc_rus': 11.495211250964203,
 'index_fk_rus': 13.65274483499585,
 'letters': 244494,
 'n_complex_words': 5651,
 'n_sentences': 3424,
 'n_simple_words': 28691,
 'n_syllabes': 104122,
 'n_words': 36251,
 'spaces': 37192,
 'wsyllabes': {'1': 5673,
  '10': 18,
  '11': 1,
  '12': 1,
  '2': 7876,
  '3': 8893,
  '4': 6249,
  '5': 3637,
  '6': 1491,
  '7': 361,
  '8': 117,
  '9': 25}}

#### Cosine similarity

In [ ]:
calc_cos_sim(data_test, model, tok, 'INPUT:source', 'ru_paraphraser', 'cos_sim_ru_paraphraser')
data_test.cos_sim_ru_paraphraser.mean()

0.9436661657985

#### Grammar

In [ ]:
errors = get_mistakes_summary(data_test, 'ru_paraphraser')
errors

{'CASING': 26,
 'GRAMMAR': 32,
 'LOGIC': 27,
 'PUNCTUATION': 14,
 'STYLE': 4,
 'TYPOGRAPHY': 141,
 'TYPOS': 598}

### NEW english data FKGL

In [ ]:
# https://github.com/infoculture/plainrussian
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from math import sqrt
import csv


from numpy import mean, arange


# Russian sounds and characters
# RU_CONSONANTS_LOW = [u'к', u'п', u'с', u'т', u'ф', u'х', u'ц', u'ч', u'ш', u'щ']
# RU_CONSONANTS_HIGH = [u'б', u'в', u'г', u'д', u'ж', u'з']
# RU_CONSONANTS_SONOR = [u'л', u'м', u'н', u'р']
# RU_CONSONANTS_YET = [u'й']

EN_CONSONANTS = [u'b', u'c', u'd', u'f', u'g', u'h', u'j', u'k', u'l', u'm', u'n', u'p', u'r', u's', u't', u'q', u'x', u'w', u'v', u'z']
EN_VOWELS = [u'a', u'e', u'i', u'y', u'o', u'u']

SENTENCE_SPLITTERS = [u'.', u'?', u'!']
EN_LETTERS = EN_CONSONANTS + EN_VOWELS
SPACES = [u' ', u'\t']

# List of prepared texts

GRADE_TEXT = {
    1: u'1 - 3-й класс (возраст примерно: 6-8 лет)',
    2: u'1 - 3-й класс (возраст примерно: 6-8 лет)',
    3: u'1 - 3-й класс (возраст примерно: 6-8 лет)',
    4: u'4 - 6-й класс (возраст примерно: 9-11 лет)',
    5: u'4 - 6-й класс (возраст примерно: 9-11 лет)',
    6: u'4 - 6-й класс (возраст примерно: 9-11 лет)',
    7: u'7 - 9-й класс (возраст примерно: 12-14 лет)',
    8: u'7 - 9-й класс (возраст примерно: 12-14 лет)',
    9: u'7 - 9-й класс (возраст примерно: 12-14 лет)',
    10: u'10 - 11-й класс (возраст примерно: 15-16 лет)',
    11: u'10 - 11-й класс (возраст примерно: 15-16 лет)',
    12: u'1 - 3 курсы ВУЗа (возраст примерно: 17-19 лет)',
    13: u'1 - 3 курсы ВУЗа (возраст примерно: 17-19 лет)',
    14: u'1 - 3 курсы ВУЗа (возраст примерно: 17-19 лет)',
    15: u'4 - 6 курсы ВУЗа (возраст примерно: 20-22 лет)',
    16: u'4 - 6 курсы ВУЗа (возраст примерно: 20-22 лет)',
    17: u'4 - 6 курсы ВУЗа (возраст примерно: 20-22 лет)',
}

POST_GRADE_TEXT_18_24 = u'Аспирантура, второе высшее образование, phD'


def calc_SMOG(n_psyl, n_sent):
    """Метрика SMOG для английского языка"""
    n = 1.0430 * sqrt((float(30.0) / n_sent) * n_psyl) + 3.1291
    return n

def calc_Gunning_fog(n_psyl, n_words, n_sent):
    """Метрика Gunning fog для английского языка"""
    n = 0.4 * ((float(n_words)/ n_sent) + 100 * (float(n_psyl) / n_words))
    return n

def calc_Dale_Chale(n_psyl, n_words, n_sent):
    """Метрика Dale Chale для английского языка"""
    n = 0.1579 * (100.0 * n_psyl / n_words) + 0.0496 * (float(n_words) / n_sent)
    return n

def calc_Flesh_Kincaid(n_syllabes, n_words, n_sent):
    """Метрика Flesh Kincaid для английского языка"""
    n = 206.835 - 1.015 * (float(n_words) / n_sent) - 84.6 * (float(n_syllabes) / n_words)
    return n


def calc_Flesh_Kincaid_rus(n_syllabes, n_words, n_sent):
    """Метрика Flesh Kincaid для русского языка"""
    n = 220.755 - 1.315 * (float(n_words) / n_sent) - 50.1 * (float(n_syllabes) / n_words)
    return n


# The variant of the FC formula for the Russian language is as follows:FK = (0.5 x average sentence length) +
# (8.4 x average number of syllables in a word) - 15.59

def calc_Flesh_Kincaid_Grade(n_syllabes, n_words, n_sent):
    """Метрика Flesh Kincaid Grade для русского языка"""
#    n = 0.59 * (float(n_words) / n_sent) + 6.2 * (float(n_syllabes) / n_words) - 16.59
    n = 0.39 * (float(n_words) / n_sent) + 11.8 * (float(n_syllabes) / n_words) - 15.59
    return n

def calc_Flesh_Kincaid_Grade_rus(n_syllabes, n_words, n_sent):
    """Метрика Flesh Kincaid Grade для русского языка"""
#    n = 0.59 * (float(n_words) / n_sent) + 6.2 * (float(n_syllabes) / n_words) - 16.59
    n = 0.5 * (float(n_words) / n_sent) + 8.4 * (float(n_syllabes) / n_words) - 15.59
    return n



def calc_Flesh_Kincaid_Grade_rus_adapted(n_syllabes, n_words, n_sent, X, Y, Z):
    """Метрика Flesh Kincaid Grade для русского языка с параметрами"""
#    n = 0.59 * (float(n_words) / n_sent) + 6.2 * (float(n_syllabes) / n_words) - 16.59
    if n_words == 0 or n_sent == 0: return 0
    n = X * (float(n_words) / n_sent) + Y * (float(n_syllabes) / n_words) - Z
    return n


#X_GRADE = 0.186
#Y_GRADE = 7.21
#Z_GRADE = 15.443

# Flesh Kinkaid Grade константы. Подробнее http://en.wikipedia.org/wiki/Flesch%E2%80%93Kincaid_readability_tests
FLG_X_GRADE = 0.318
FLG_Y_GRADE = 14.2
FLG_Z_GRADE = 30.5

def calc_Flesh_Kincaid_Grade_rus_flex(n_syllabes, n_words, n_sent):
    """Метрика Flesh Kincaid Grade для русского языка с константными параметрами"""
    if n_words == 0 or n_sent == 0: return 0
    n = FLG_X_GRADE * (float(n_words) / n_sent) + FLG_Y_GRADE * (float(n_syllabes) / n_words) - FLG_Z_GRADE
    return n


# Coleman Liau константы. Подробнее http://en.wikipedia.org/wiki/Coleman%E2%80%93Liau_index

CLI_X_GRADE = 0.055
CLI_Y_GRADE = 0.35
CLI_Z_GRADE = 20.33


def calc_Coleman_Liau_index_adapted(n_letters, n_words, n_sent, x, y, z):
    """ Метрика Coleman Liau для русского языка с адаптированными параметрами """
    if n_words == 0: return 0
    n = x * (n_letters * (100.0 / n_words)) - y * (n_sent * (100.0 / n_words)) - z
    return n

def calc_Coleman_Liau_index(n_letters, n_words, n_sent):
    """ Метрика Coleman Liau для русского языка с константными параметрами """
    if n_words == 0: return 0
    n = CLI_X_GRADE * (n_letters * (100.0 / n_words)) - CLI_Y_GRADE * (n_sent * (100.0 / n_words)) - CLI_Z_GRADE
    return n


# Константы SMOG Index http://en.wikipedia.org/wiki/SMOG
SMOG_X_GRADE = 1.1
SMOG_Y_GRADE = 64.6
SMOG_Z_GRADE = 0.05

def calc_SMOG_index(n_psyl, n_sent):
    """Метрика SMOG для русского языка с константными параментрами"""
    n = SMOG_X_GRADE * sqrt((float(SMOG_Y_GRADE) / n_sent) * n_psyl) + SMOG_Z_GRADE
    return n

def calc_SMOG_index_adapted(n_psyl, n_sent, x, y, z):
    """Метрика SMOG для русского языка адаптированная с коэффициентами"""
    n = x * sqrt((float(y) / n_sent) * n_psyl) + z
    return n

DC_X_GRADE = 0.552
DC_Y_GRADE = 0.273

def calc_Dale_Chale_index(n_psyl, n_words, n_sent):
    """Метрика Dale Chale для русского языка с константным параметрами"""
    n = DC_X_GRADE * (100.0 * n_psyl / n_words) + DC_Y_GRADE * (float(n_words) / n_sent)
    return n


def calc_Dale_Chale_adapted(n_psyl, n_words, n_sent, x, y):
    """Метрика Dale Chale для русского языка с адаптированными параметрами"""
    n = x * (100.0 * n_psyl / n_words) + y * (float(n_words) / n_sent)
    return n

ARI_X_GRADE = 6.26
ARI_Y_GRADE = 0.2805
ARI_Z_GRADE = 31.04


def calc_ARI_index_adapted(n_letters, n_words, n_sent, x, y, z):
    """ Метрика Automated Readability Index (ARI) для русского языка с адаптированными параметрами """
    if n_words == 0 or n_sent == 0: return 0
    n = x * (float(n_letters) / n_words) + y * (float(n_words) / n_sent) - z
    return n

def calc_ARI_index(n_letters, n_words, n_sent):
    """ Метрика Automated Readability Index (ARI) для русского языка с константными параметрами """
    if n_words == 0 or n_sent == 0: return 0
    n = ARI_X_GRADE * (float(n_letters) / n_words) + ARI_Y_GRADE * (float(n_words) / n_sent) - ARI_Z_GRADE
    return n


def load_words(filename):
    """Load words from filename"""
    words = []
    f = open(filename, 'r')
    for l in f:
        words.append(l.strip().decode('utf8'))
    f.close()
    return words

#FAM_WORDS = load_words('1norm50000.txt')

bad_chars = '(){}<>"\'!?,.:;'


def calc_text_metrics(filename, verbose=True):
    """Расчет метрик"""
    f = open(filename, 'r')
    text = f.read().decode('utf8')    
    f.close()
    return calc_readability_metrics(text, verbose)


# Number of syllabes for long words
COMPLEX_SYL_FACTOR = 4

def calc_readability_metrics(text, verbose=True):
    sentences = 0
    chars = 0
    spaces = 0
    letters = 0
    syllabes = 0
    words = 0
    complex_words = 0
    simple_words = 0
    wsyllabes = {}

    wordStart = False
    for l in text.splitlines():
        chars += len(l)
#        l = l.decode('utf8')
        for ch in l:
            if ch in SENTENCE_SPLITTERS:
                sentences += 1
            if ch in SPACES:
                spaces += 1

        for w in l.split():
            has_syl = False
            wsyl = 0
#            if len(w) > 1: words += 1
            for ch in w:
                if ch in EN_LETTERS:
                    letters += 1
                if ch in EN_VOWELS:
                    syllabes += 1
                    has_syl = True
                    wsyl += 1
            if wsyl > COMPLEX_SYL_FACTOR:
                complex_words += 1
            elif wsyl < COMPLEX_SYL_FACTOR+1 and wsyl > 0:
                simple_words += 1
            if has_syl:
                words += 1
                v = wsyllabes.get(str(wsyl), 0)
                wsyllabes[str(wsyl)] = v + 1
    metrics = {'c_share': float(complex_words) * 100 / words if words > 0 else 0,
               'avg_slen' : float(words) / sentences if sentences > 0 else 0,
               'avg_syl' : float(syllabes) / words if words > 0 else 0,
               'n_syllabes': syllabes,
               'n_words' : words,
               'n_sentences': sentences,
               'n_complex_words': complex_words,
               'n_simple_words' : simple_words,
               'chars': chars,
               'letters' : letters,
               'spaces' : spaces,
               'index_fk': calc_Flesh_Kincaid_Grade(syllabes, words, sentences),  #calc_Flesh_Kincaid_Grade_rus_flex
               'wsyllabes' : wsyllabes
    }
    del text
    return metrics

In [ ]:
calc_readability_metrics(' '.join(list(data_test.en_en.values)))

{'avg_slen': 19.29485049833887,
 'avg_syl': 2.0208629274102132,
 'c_share': 4.674787998794714,
 'chars': 445593,
 'index_fk': 15.781174237792676,
 'letters': 347098,
 'n_complex_words': 3258,
 'n_sentences': 3612,
 'n_simple_words': 66435,
 'n_syllabes': 140840,
 'n_words': 69693,
 'spaces': 79167,
 'wsyllabes': {'1': 32475,
  '10': 6,
  '2': 16848,
  '3': 11056,
  '4': 6056,
  '5': 2448,
  '6': 694,
  '7': 72,
  '8': 33,
  '9': 5}}

In [ ]:
calc_readability_metrics(' '.join(list(data_test.en_en_filtered.values)))

{'avg_slen': 12.83673469387755,
 'avg_syl': 2.034862593686123,
 'c_share': 4.649102884397002,
 'chars': 282865,
 'index_fk': 13.427705136108496,
 'letters': 219784,
 'n_complex_words': 2047,
 'n_sentences': 3430,
 'n_simple_words': 41983,
 'n_syllabes': 89595,
 'n_words': 44030,
 'spaces': 49875,
 'wsyllabes': {'1': 20009,
  '2': 10913,
  '3': 7270,
  '4': 3791,
  '5': 1569,
  '6': 423,
  '7': 37,
  '8': 18}}

In [ ]:
with open('/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_en_en_filtered.hyp', 'r') as f:
  sentences = [i.strip() for i in f.readlines()]

lt = list()
st = set()
for i in sentences:
  if i not in st:
    lt.append(i)
    st.add(i)

with open('/content/model_prediction_en_en_filtered.hyp', 'w') as f:
  for i in lt:
    f.write(i+'\n')

In [ ]:
! easse evaluate \
--test_set custom \
--metrics fkgl \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction_en_en_filtered.hyp -q

{'fkgl': 8.521, 'quality_estimation': {'Compression ratio': 0.591, 'Sentence splits': 0.99, 'Levenshtein similarity': 0.133, 'Exact copies': 0.0, 'Additions proportion': 0.635, 'Deletions proportion': 0.911, 'Lexical complexity score': 8.217}}


In [ ]:
with open('/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_en_en.hyp', 'r') as f:
  sentences = [i.strip() for i in f.readlines()]

lt = list()
st = set()
for i in sentences:
  if i not in st:
    lt.append(i)
    st.add(i)

with open('/content/model_prediction_en_en.hyp', 'w') as f:
  for i in lt:
    f.write(i+'\n')

In [ ]:
! easse evaluate \
--test_set custom \
--metrics fkgl \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction_en_en.hyp -q

{'fkgl': 11.159, 'quality_estimation': {'Compression ratio': 0.929, 'Sentence splits': 1.042, 'Levenshtein similarity': 0.156, 'Exact copies': 0.0, 'Additions proportion': 0.869, 'Deletions proportion': 0.771, 'Lexical complexity score': 8.294}}


In [ ]:
import os
os.path.isdir('/content/drive/MyDrive/checkpoints_ru_ru_add')

False

## Evaluation of amosel trained on mix of ru and ParaPhraser

### SARI

In [ ]:
with open('/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_wiki_15_filtered_epochs.hyp', 'r') as f:
  sentences = [i.strip() for i in f.readlines()]

lt = list()
st = set()
for i in sentences:
  if i not in st:
    lt.append(i)
    st.add(i)

with open('/content/model_prediction_wiki_15_filtered_epochs.test.hyp', 'w') as f:
  for i in lt:
    f.write(i+'\n')

In [ ]:
! easse evaluate \
--test_set custom \
--metrics sari \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction_wiki_15_filtered_epochs.test.hyp -q

[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
{'sari': 34.312, 'quality_estimation': {'Compression ratio': 0.683, 'Sentence splits': 0.997, 'Levenshtein similarity': 0.629, 'Exact copies': 0.007, 'Additions proportion': 0.216, 'Deletions proportion': 0.546, 'Lexical complexity score': 10.732}}


### BLEU

In [ ]:
# make lists of src, dst
src_column, trg_column = "ru_+_paraphraser", "OUTPUT:output"
data_dict = OrderedDict()
for ind, row in data_test.iterrows():
        source_sentence = row[src_column]
        reference_sentence = row[trg_column]
        if data_dict.get(source_sentence) is None:
            data_dict[source_sentence] = []
        data_dict[source_sentence].append(reference_sentence)
max_num_reference_sentences = max([len(x) for x in data_dict.values()])

src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
for i, j in enumerate(src):
   if len(dst[i]) < max_num_reference_sentences:
    dst[i] = dst[i] + [dst[i][0]]*(max_num_reference_sentences - len(dst[i]))

refs = []
for j in range(len(dst[1])):
  ref = []
  for i in dst:
    ref.append(i[j])
  refs.append(ref)
  
corpus_bleu(src, refs)


36.640087511601344

In [ ]:
refs = []
for i, j in data_dict.items():
  refs.append(j)

nltk.translate.bleu_score.corpus_bleu(refs, src)

0.7649725573817129

## FKLG

In [ ]:
calc_readability_metrics(' '.join(list(data_test['ru_+_paraphraser'])))

{'avg_slen': 11.279015918958033,
 'avg_syl': 2.870127537273217,
 'c_share': 15.66373271061613,
 'chars': 316048,
 'index_SMOG_rus': 11.801469582022179,
 'index_ari_rus': 14.284347506030898,
 'index_cl_rus': 13.608938643537165,
 'index_dc_rus': 11.725551802135648,
 'index_fk_rus': 13.842538091508338,
 'letters': 262453,
 'n_complex_words': 6104,
 'n_sentences': 3455,
 'n_simple_words': 30856,
 'n_syllabes': 111846,
 'n_words': 38969,
 'spaces': 39879,
 'wsyllabes': {'1': 6047,
  '10': 30,
  '11': 6,
  '12': 1,
  '2': 8778,
  '3': 9453,
  '4': 6578,
  '5': 3917,
  '6': 1586,
  '7': 441,
  '8': 101,
  '9': 22}}

## Cosine similarity

In [ ]:
calc_cos_sim(data_test, model, tok, 'INPUT:source', 'ru_+_paraphraser', 'cos_sim_ru_p')
data_test.cos_sim_ru_p.mean()


0.9596704395637747

## Grammar

In [ ]:
errors = get_mistakes_summary(data_test, 'ru_+_paraphraser')
errors

{'CASING': 24,
 'GRAMMAR': 20,
 'LOGIC': 27,
 'PUNCTUATION': 18,
 'STYLE': 6,
 'TYPOGRAPHY': 194,
 'TYPOS': 581}

## Evaluation of amosel trained with control tokens


### SARI

In [ ]:
with open('/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_con_tok_38sari.hyp', 'r') as f:
  sentences = [i.strip() for i in f.readlines()]

lt = list()
st = set()
for i in sentences:
  if i not in st:
    lt.append(i)
    st.add(i)

with open('/content/model_prediction_con_tok_38sari.test.hyp', 'w') as f:
  for i in lt:
    f.write(i+'\n')


In [ ]:
! easse evaluate \
--test_set custom \
--metrics sari \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction_con_tok_38sari.test.hyp -q


{'sari': 38.823, 'quality_estimation': {'Compression ratio': 0.932, 'Sentence splits': 1.372, 'Levenshtein similarity': 0.464, 'Exact copies': 0.0, 'Additions proportion': 0.593, 'Deletions proportion': 0.585, 'Lexical complexity score': 10.489}}


### BLEU

In [ ]:
# make lists of src, dst
src_column, trg_column = "con_tok", "OUTPUT:output"
data_dict = OrderedDict()
for ind, row in data_test.iterrows():
        source_sentence = row[src_column]
        reference_sentence = row[trg_column]
        if data_dict.get(source_sentence) is None:
            data_dict[source_sentence] = []
        data_dict[source_sentence].append(reference_sentence)
max_num_reference_sentences = max([len(x) for x in data_dict.values()])

src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
for i, j in enumerate(src):
   if len(dst[i]) < max_num_reference_sentences:
    dst[i] = dst[i] + [dst[i][0]]*(max_num_reference_sentences - len(dst[i]))

refs = []
for j in range(len(dst[1])):
  ref = []
  for i in dst:
    ref.append(i[j])
  refs.append(ref)
  
corpus_bleu(src, refs)

8.697924628913844

In [ ]:
sacrebleu.corpus_bleu(src, refs)

BLEU = 8.70 33.5/11.8/5.6/2.6 (BP = 1.000 ratio = 1.254 hyp_len = 21478 ref_len = 17125)

In [ ]:
refs = []
for i, j in data_dict.items():
  refs.append(j)

nltk.translate.bleu_score.corpus_bleu(refs, src)

0.46781873246085204

## FKLG

In [ ]:
calc_readability_metrics(' '.join(list(data_test.con_tok)))


{'avg_slen': 12.230621805792163,
 'avg_syl': 2.653991468616697,
 'c_share': 12.27474536432489,
 'chars': 439010,
 'index_SMOG_rus': 10.88276840793238,
 'index_ari_rus': 10.655763674381411,
 'index_cl_rus': 10.427804474623489,
 'index_dc_rus': 10.1146191940886,
 'index_fk_rus': 11.076016588599003,
 'letters': 351079,
 'n_complex_words': 7050,
 'n_sentences': 4696,
 'n_simple_words': 47192,
 'n_syllabes': 152432,
 'n_words': 57435,
 'spaces': 60392,
 'wsyllabes': {'1': 10632,
  '10': 26,
  '11': 3,
  '12': 1,
  '2': 15318,
  '3': 12747,
  '4': 8495,
  '5': 4329,
  '6': 2069,
  '7': 452,
  '8': 115,
  '9': 55}}

## Cosine similarity

In [ ]:
calc_cos_sim(data_test, model, tok, 'INPUT:source', 'con_tok', 'cos_sim_con_tok')
data_test.cos_sim_con_tok.mean()

0.9382814637314763

### Grammar

In [ ]:
errors = get_mistakes_summary(data_test, 'con_tok')
errors

{'CASING': 21,
 'GRAMMAR': 41,
 'LOGIC': 6,
 'MISC': 12,
 'PUNCTUATION': 40,
 'STYLE': 46,
 'TYPOGRAPHY': 837,
 'TYPOS': 1058}